In [1]:
# dbutils.library.install("dbfs:/FileStore/jars/52c127a5_5f6b_4286_84e2_6aee83ebb483-MarketingMix_0_2_py3_7-a49e4.egg")
# dbutils.library.install("dbfs:/FileStore/jars/e850cc88_b78f_41dd_b941_4a7bdd4bdef3-MarketingMix_1_0_py3_7-08c12.egg")
dbutils.library.install("dbfs:/FileStore/jars/f5c4bd92_8e94_4275_a120_0af2a5833f20-MarketingMix_1_1_py3_7-3eb73.egg")
dbutils.library.restartPython()

In [2]:
import numpy as np
import pandas as pd
from MarketingMixModel import MarketingMixModel

In [3]:
def convert_to_var(query, var_name):
  
#   if "select " not in query:
#     print("Query must be in form of select statement")
#     return
  
#   if var_name not in query:
#     print("Select statement must contain input variable name")
#     return
  
  var_df = spark.sql(query)

  # Keep it as a dictionary to keep it flexible in case of later development
  var_dict = {}
  for column_name in var_df.columns:
    var_dict[column_name] = [x[column_name] for x in var_df.rdd.collect()][0]
    
  return var_dict[var_name]  

In [4]:
def mmm_loop(min_month, max_month, country):

  mmm_target =["new_total_bkgs","rpt_total_bkgs","order_count_new_bc","order_count_new_cons","order_count_new_other","order_count_new_plt","order_count_new_ppag","order_count_new_signage","order_count_new_mm"]
  
  log_target = ["new_total_bkgs_log","rpt_total_bkgs_log"]
  
  final_std_df = pd.DataFrame(columns=['l_psnb', 'l_psb', 'l_disp', 'l_tv_radio', 'l_aff', 'l_email', 'discount_new_log', 'l_psnb_clicks', 'l_psnb_imps', 'metric_value','metric','month_start_date','country'])
  
  final_irf_df_1 = pd.DataFrame(columns=['psnb_total_spend_usd', 'psb_spend', 'display_spend', 'tv_spend', 'affiliate_spend', 'psnb_total_clicks', 'psb_clicks','display_clicks','period','figure','response','month_start_date'])
  
  final_irf_df = pd.DataFrame(columns=['psnb_total_spend_usd', 'psb_spend', 'display_spend', 'tv_spend', 'affiliate_spend', 'email_sends_promotional', 'discount_new', 'psnb_total_clicks', 'psnb_total_impressions', 'metric', 'period','metric_value','figure','response','month_start_date'])
  
  final_irf_df_pivot = pd.DataFrame(columns=['psnb_total_spend_usd', 'psb_spend', 'display_spend', 'tv_spend', 'affiliate_spend', 'email_sends_promotional', 'discount_new', 'psnb_total_clicks', 'psnb_total_impressions','metric', 'period','metric_value','figure','response','month_start_date'])
  
  final_irf_df_1_pivot = pd.DataFrame(columns=['psnb_total_spend_usd', 'psb_spend', 'display_spend', 'tv_spend', 'affiliate_spend', 'psnb_total_clicks', 'psb_clicks','display_clicks', 'period','figure','response','month_start_date'])
  
  final_df = pd.DataFrame(columns=['psnb', 'psb', 'display', 'tv_radio', 'affiliate', 'email', 'discount', 'psnb_total_clicks', 'psnb_total_impressions', 'momentum','metric','date','country','total_revenue','exog_adj'])
  
  while min_month <= max_month :
    min_date = str(convert_to_var("select add_months('"+ min_month + "', -11) as date", "date"))
    max_date = str(convert_to_var("select date_add( add_months('"+ min_month + "', 1), -1 ) as date", "date"))
#     max_date = min_month
    print(min_date)
    print(max_date)
    print(min_month)
    
  #   else : print(end)  
#     query = ("refresh table rswaddle_"+ country + "_mmm_overall_data_clean")
    
    query = """
            select *, (tv_spend + radio_spend) as tv_radio_spend from rswaddle_""" + country + """_mmm_overall_data_clean
          """ 
    df = spark.sql(query)
    
    log_query = """
            select *, log(tv_spend + radio_spend + 1) as tv_radio_spend_log, log(new_total_bkgs + 1) as new_total_bkgs_log, log(rpt_total_bkgs + 1) as rpt_total_bkgs_log, log(discount_new + 1) as discount_new_log, log(discount_rpt + 1) as discount_rpt_log, log(psnb_total_clicks + 1) as psnb_total_clicks_log, log(psnb_total_impressions + 1) as psnb_total_impressions_log, log(email_sends_promotional + 1) as email_sends_promotional_log, log(affiliate_spend + 1) as affiliate_spend_log, log(display_spend + 1) as display_spend_log, log(psb_spend + 1) as psb_spend_log, log(psnb_total_spend_usd + 1) as psnb_total_spend_usd_log  from rswaddle_""" + country + """_mmm_overall_data_clean
          """ 
    log_df = spark.sql(log_query)

    MMM_log = MarketingMixModel.MarketingMixModel(spark_df=log_df, start_date= min_date , end_date= max_date, covid_st='2020-03-14', covid_ed='2020-09-01')
    
#     MMM_chnl_inter = MarketingMixModel.MarketingMixModel(spark_df=df, start_date= min_date , end_date= max_date, covid_st='2020-03-14', covid_ed='2020-09-01')
  
    MMM = MarketingMixModel.MarketingMixModel(spark_df=df, start_date= min_date , end_date= max_date, covid_st='2020-03-14', covid_ed='2020-09-01') 
    
    if min_month >= '2020-03-01' :
      MMM.fit(endog=['psnb_total_spend_usd', 'psb_spend', 'display_spend', 'tv_spend', 'affiliate_spend', 'psnb_total_clicks', 'psb_clicks','display_clicks'], exog=['c'], lag=2, week_day=True, is_covid=True)
      
#         exog = MMM.exogenous_impact(revs=mmm_value, week_day=True)
#         exog = exog[-1]
    else:
      MMM.fit(endog=['psnb_total_spend_usd', 'psb_spend', 'display_spend', 'tv_spend', 'affiliate_spend', 'psnb_total_clicks', 'psb_clicks','display_clicks'], exog=['c'], lag=2, week_day=True, is_covid=False)

#       exog = MMM.exogenous_impact(revs=mmm_value, week_day=True)

    irf_ci = MMM.g_irf(20)
#     print(irf_ci)
    
    period_1 = 1

    while period_1 <= 19 :

      irf_ci_df = pd.DataFrame({ 'psnb_total_spend_usd': irf_ci[period_1,0],'psb_spend': irf_ci[period_1,1], 'display_spend': irf_ci[period_1,2],'tv_spend': irf_ci[period_1,3],'affiliate_spend': irf_ci[period_1,4],'psnb_total_clicks': irf_ci[period_1,5],'psb_clicks': irf_ci[period_1,6],'display_clicks': irf_ci[period_1,7]})

      irf_ci_df['period']= period_1
#         irf_ci_df['metric'] = mmm_value
      irf_ci_df['figure'] = 'value'
  
      irf_ci_df = irf_ci_df.rename(index={0: "psnb_total_spend_usd", 1: "psb_spend", 2: "display_spend", 3: "tv_spend", 4: "affiliate_spend", 5: "psnb_total_clicks", 6: "psb_clicks", 7: "display_clicks"})
  
      irf_ci_df['response'] = irf_ci_df.index
    
#       irf_ci_df['response'] = {"psnb_total_spend_usd", "psb_spend", "display_spend", "tv_spend", "affiliate_spend", "psnb_total_clicks", "psnb_total_impressions"}
      
      irf_ci_df['month_start_date'] = min_month

      print(irf_ci_df)
      print(type(irf_ci_df))
      final_irf_df_1 = final_irf_df_1.append(irf_ci_df, ignore_index=True)

      period_1 = period_1 + 1
    
    for mmm_value in mmm_target:
      if mmm_value == 'new_total_bkgs':
        mmm_discount = 'discount_new'
      elif mmm_value == 'rpt_total_bkgs':
        mmm_discount = 'discount_rpt'
      elif mmm_value == 'order_count_new_bc':
        mmm_discount = 'discount_new_bc'  
      elif mmm_value == 'order_count_new_cons':
        mmm_discount = 'discount_new_cons'  
      elif mmm_value == 'order_count_new_other':
        mmm_discount = 'discount_new_other'  
      elif mmm_value == 'order_count_new_plt':
        mmm_discount = 'discount_new_plt'  
      elif mmm_value == 'order_count_new_ppag':
        mmm_discount = 'discount_new_ppag'  
      elif mmm_value == 'order_count_new_signage':
        mmm_discount = 'discount_new_signage'  
      elif mmm_value == 'order_count_new_mm':
        mmm_discount = 'discount_new_mm' 
      
      if min_month >= '2020-03-01' :
        MMM.fit(endog=['psnb_total_spend_usd','psb_spend','display_spend','tv_radio_spend','affiliate_spend','email_sends_promotional',mmm_discount,'psnb_total_clicks','psnb_total_impressions',mmm_value ], exog=['c'], lag=2, week_day=True, is_covid=True)
      
        exog = MMM.exogenous_impact(revs=mmm_value, week_day=True)
        exog = exog[-1]
      else:
        MMM.fit(endog=['psnb_total_spend_usd','psb_spend','display_spend','tv_radio_spend','affiliate_spend','email_sends_promotional',mmm_discount,'psnb_total_clicks','psnb_total_impressions',mmm_value ], exog=['c'], lag=2, week_day=True, is_covid=False)
      
        exog = MMM.exogenous_impact(revs=mmm_value, week_day=True)
      
      print(exog)
      print(mmm_value)
      print(mmm_discount)
#       print(MMM.g_irf(20))
      print(MMM.g_irf(20)[-1])
      print(MMM.g_fevd(20)[-1])
      fevd = MMM.g_fevd(20)
      fevd_df = pd.DataFrame({ 'psnb_total_clicks': fevd[19,7], 'psnb_total_impressions': fevd[19,8], mmm_value : fevd[19,9]})
      fevd_df = fevd_df.rename(index={0: "psnb", 1: "psb", 2: "display", 3: "tv_radio", 4: "affiliate", 5: "email", 6: "discount", 7: "psnb_total_clicks", 8: "psnb_total_impressions", 9: "momentum"})

      fevd_df['psnb_total_clicks'].iat[7] = 0 
      fevd_df['psnb_total_clicks'].iat[8] = 0 
      fevd_df['psnb_total_impressions'].iat[7] = 0 
      fevd_df['psnb_total_impressions'].iat[8] = 0 

      total_psnb_imps = fevd_df['psnb_total_impressions'].sum()
      total_psnb_clicks = fevd_df['psnb_total_clicks'].sum()

      new_total_bkgs_psnb_clicks_contribution  = fevd_df[mmm_value].iat[7] 
      new_total_bkgs_psnb_imps_contribution = fevd_df[mmm_value].iat[8]

      fevd_df['total_combined_contribution'] = fevd_df['psnb_total_impressions']*new_total_bkgs_psnb_imps_contribution/total_psnb_imps +       fevd_df['psnb_total_clicks']*new_total_bkgs_psnb_clicks_contribution/total_psnb_clicks + fevd_df[mmm_value]

      fevd_df['total_combined_contribution'].iat[7] = 0 
      fevd_df['total_combined_contribution'].iat[8] = 0 
      
#       exog_variable = MMM.exogenous_impact(revs=mmm_value, week_day=True)
#       print(exog_variable)
      
      print(fevd_df)
      
      query_2 = """
            select sum(""" + mmm_value + """) as metric_total from rswaddle_""" + country + """_mmm_overall_data_clean where date >= '""" + min_month + """' and date <=  '""" + max_date + """'
          """  
      metric_calc = spark.sql(query_2)
      metric_calc = metric_calc.select('metric_total').collect()
      metric_calc = metric_calc[0].metric_total
      print(metric_calc)
    
  #     fevd_df_2 = fevd_df['total_combined_contribution']
      fevd_df = np.transpose(fevd_df) 
      fevd_df = fevd_df.iloc[3]
      fevd_df['metric'] = mmm_value
      fevd_df['date'] = min_month
      fevd_df['country'] = country
      fevd_df['total_revenue'] = metric_calc
      fevd_df['exog_adj'] = exog
  #     fevd_df = pd.DataFrame(data = fevd_df)
#       fevd_df = fevd_df.values.tolist()
#       fevd_df = fevd_df.reset_index()
      print(fevd_df)
      final_df = final_df.append(fevd_df)
      final_df.reset_index()
      print(final_df)
      print(min_month)
      
    
    for log_value in log_target:
      if log_value == 'new_total_bkgs_log':
        log_discount = 'discount_new_log'
      elif log_value == 'rpt_total_bkgs_log':
        log_discount = 'discount_rpt_log' 
        
      if min_month >= '2020-03-01' :
        MMM_log.fit(endog=['psnb_total_spend_usd_log','psb_spend_log','display_spend_log','tv_radio_spend_log','affiliate_spend_log','email_sends_promotional_log',log_discount,'psnb_total_clicks_log','psnb_total_impressions_log',log_value ], exog=['c'], lag=2, week_day=True, is_covid=True)
      
        exog_log = MMM_log.exogenous_impact(revs=log_value, week_day=True)
        exog_log = exog_log[-1]
      else:
        MMM_log.fit(endog=['psnb_total_spend_usd_log','psb_spend_log','display_spend_log','tv_radio_spend_log','affiliate_spend_log','email_sends_promotional_log',log_discount,'psnb_total_clicks_log','psnb_total_impressions_log',log_value ], exog=['c'], lag=2, week_day=True, is_covid=False)
      
        exog_log = MMM_log.exogenous_impact(revs=log_value, week_day=True) 
        
      log_irf = MMM_log.g_irf(20)
      print(log_irf)
      
      std_df = MMM_log.get_std()
      
      print(std_df)
      
      std_df = pd.DataFrame({ 'l_psnb': std_df.psnb_total_spend_usd_log,'l_psb': std_df.psb_spend_log, 'l_disp': std_df.display_spend_log,'l_tv_radio': std_df.tv_radio_spend_log,'l_aff': std_df.affiliate_spend_log,'l_email': std_df.email_sends_promotional_log,'discount_new_log': std_df[6],'l_psnb_clicks': std_df.psnb_total_clicks_log, 'l_psnb_imps': std_df.psnb_total_impressions_log, 'metric_value' : std_df[9]}, index=[0])
      
      std_df['metric'] = log_value
      std_df['month_start_date'] = min_month
      std_df['country'] = country
      
      final_std_df = final_std_df.append(std_df, ignore_index=True)
      
      period = 1
      
      while period <= 19 :

        log_irf_df = pd.DataFrame({ 'psnb_total_spend_usd': log_irf[period,0],'psb_spend': log_irf[period,1], 'display_spend': log_irf[period,2],'tv_spend': log_irf[period,3],'affiliate_spend': log_irf[period,4],'email_sends_promotional': log_irf[period,5],'discount_new': log_irf[period,6],'psnb_total_clicks': log_irf[period,7], 'psnb_total_impressions': log_irf[period,8], 'metric_value' : log_irf[period,9]})

        log_irf_df['period']= period
        log_irf_df['metric'] = log_value
        log_irf_df['figure'] = 'value'
        
        log_irf_df = log_irf_df.rename(index={0: "psnb_total_spend_usd", 1: "psb_spend", 2: "display_spend", 3: "tv_spend", 4: "affiliate_spend", 5: "email_sends_promotional", 6: "discount_new", 7: "psnb_total_clicks", 8: "psnb_total_impressions", 9: "metric_value"})
  
        log_irf_df['response'] = log_irf_df.index
  
#         log_irf_df['response'] = {"psnb_total_spend_usd", "psb_spend", "display_spend", "tv_spend", "affiliate_spend", "email_sends_promotional", "discount_new", "psnb_total_clicks", "psnb_total_impressions", "metric_value"}
    
        log_irf_df['month_start_date'] = min_month

        print(log_irf_df)
        print(type(log_irf_df))
        final_irf_df = final_irf_df.append(log_irf_df, ignore_index=True)

        period = period + 1
  
      
      
    min_month = str(convert_to_var("select add_months('"+ min_month + "', 1) as date", "date"))
    print(min_month)
  
  final_df.index = range(len(final_df))
  final_df = final_df.drop(columns=[ "psnb_total_clicks", "psnb_total_impressions"])
  
#   final_irf_df_1 = final_irf_df_1[final_irf_df_1.metric_value.isin(['new_total_bkgs', 'rpt_total_bkgs'])]
  
  pivot_target = ["psnb_total_spend_usd","psb_spend","display_spend","tv_spend","affiliate_spend","email_sends_promotional","discount_new","psnb_total_clicks","psnb_total_impressions","metric_value"]
  
  
  for pivot_value in pivot_target:
    pivot_insert = pd.pivot_table(final_irf_df, values = pivot_value, index=['period', 'figure','month_start_date','metric'], columns = 'response').reset_index()
    pivot_insert['response']= pivot_value

    final_irf_df_pivot = final_irf_df_pivot.append(pivot_insert,ignore_index=False)
  
  
  pivot_ci_target = ["psnb_total_spend_usd","psb_spend","display_spend","tv_spend","affiliate_spend","psnb_total_clicks","psb_clicks","display_clicks"]
    
  for pivot_ci_value in pivot_ci_target:
    
    pivot_insert_1 = pd.pivot_table(final_irf_df_1, values = pivot_ci_value, index=['period', 'figure','month_start_date'], columns = 'response').reset_index()
    pivot_insert_1['response']= pivot_ci_value
    
#     print(pivot_insert_1)
    final_irf_df_1_pivot = final_irf_df_1_pivot.append(pivot_insert_1,ignore_index=False)
    
#   return final_irf_df, final_std_df, final_df, final_irf_df_1, final_irf_df_pivot, final_irf_df_1_pivot
  return final_std_df, final_df, final_irf_df_pivot, final_irf_df_1_pivot

In [5]:
final_std_df, final_df, final_irf_df_pivot, final_irf_df_1_pivot = mmm_loop('2019-03-01','2019-04-01', 'US')

2018-04-01
2019-03-31
2019-03-01
Model Estimation is Done, create instance for coefficient, Error and Covariance For Error
 psnb_total_spend_usd ... month_start_date
psnb_total_spend_usd 20875.552886 ... 2019-03-01
psb_spend 16546.441497 ... 2019-03-01
display_spend 6685.574028 ... 2019-03-01
tv_spend 3176.500427 ... 2019-03-01
affiliate_spend 9692.448488 ... 2019-03-01
psnb_total_clicks 19649.236388 ... 2019-03-01
psb_clicks 16403.283754 ... 2019-03-01
display_clicks 4320.475172 ... 2019-03-01

[8 rows x 12 columns]
<class 'pandas.core.frame.DataFrame'>
 psnb_total_spend_usd ... month_start_date
psnb_total_spend_usd 18110.690740 ... 2019-03-01
psb_spend 14170.589408 ... 2019-03-01
display_spend 5975.143089 ... 2019-03-01
tv_spend 4928.171540 ... 2019-03-01
affiliate_spend 8186.042804 ... 2019-03-01
psnb_total_clicks 17134.394206 ... 2019-03-01
psb_clicks 13977.120500 ... 2019-03-01
display_clicks 3663.586627 ... 2019-03-01

[8 rows x 12 columns]
<class 'pandas.core.frame.DataFrame'>
 psnb_total_spend_usd ... month_start_date
psnb_total_spend_usd 15599.456047 ... 2019-03-01
psb_spend 12229.209931 ... 2019-03-01
display_spend 5294.688259 ... 2019-03-01
tv_spend 5921.590554 ... 2019-03-01
affiliate_spend 7262.968091 ... 2019-03-01
psnb_total_clicks 14645.150475 ... 2019-03-01
psb_clicks 11832.416967 ... 2019-03-01
display_clicks 3153.146763 ... 2019-03-01

[8 rows x 12 columns]
<class 'pandas.core.frame.DataFrame'>
 psnb_total_spend_usd ... month_start_date
psnb_total_spend_usd 13581.713458 ... 2019-03-01
psb_spend 10834.556379 ... 2019-03-01
display_spend 4860.830182 ... 2019-03-01
tv_spend 6544.087188 ... 2019-03-01
affiliate_spend 6586.865151 ... 2019-03-01
psnb_total_clicks 12675.499414 ... 2019-03-01
psb_clicks 10243.796332 ... 2019-03-01
display_clicks 2888.069786 ... 2019-03-01

[8 rows x 12 columns]
<class 'pandas.core.frame.DataFrame'>
 psnb_total_spend_usd ... month_start_date
psnb_total_spend_usd 11960.171232 ... 2019-03-01
psb_spend 9745.731993 ... 2019-03-01
display_spend 4598.951357 ... 2019-03-01
tv_spend 6798.863807 ... 2019-03-01
affiliate_spend 6013.851856 ... 2019-03-01
psnb_total_clicks 11111.314871 ... 2019-03-01
psb_clicks 8982.707393 ... 2019-03-01
display_clicks 2756.614488 ... 2019-03-01

[8 rows x 12 columns]
<class 'pandas.core.frame.DataFrame'>
 psnb_total_spend_usd ... month_start_date
psnb_total_spend_usd 10635.215471 ... 2019-03-01
psb_spend 8878.291464 ... 2019-03-01
display_spend 4439.625400 ... 2019-03-01
tv_spend 6802.349992 ... 2019-03-01
affiliate_spend 5528.907017 ... 2019-03-01
psnb_total_clicks 9839.318610 ... 2019-03-01
psb_clicks 7956.289695 ... 2019-03-01
display_clicks 2670.375248 ... 2019-03-01

[8 rows x 12 columns]
<class 'pandas.core.frame.DataFrame'>
 psnb_total_spend_usd ... month_start_date
psnb_total_spend_usd 9519.723270 ... 2019-03-01
psb_spend 8151.521434 ... 2019-03-01
display_spend 4331.449428 ... 2019-03-01
tv_spend 6642.990331 ... 2019-03-01
affiliate_spend 5102.923951 ... 2019-03-01
psnb_total_clicks 8776.897060 ... 2019-03-01
psb_clicks 7090.074511 ... 2019-03-01
display_clicks 2600.029321 ... 2019-03-01

[8 rows x 12 columns]
<class 'pandas.core.frame.DataFrame'>
 psnb_total_spend_usd ... month_start_date
psnb_total_spend_usd 8558.551456 ... 2019-03-01
psb_spend 7512.117235 ... 2019-03-01
display_spend 4242.996597 ... 2019-03-01
tv_spend 6376.792752 ... 2019-03-01
affiliate_spend 4718.774117 ... 2019-03-01
psnb_total_clicks 7868.450968 ... 2019-03-01
psb_clicks 6332.988959 ... 2019-03-01
display_clicks 2532.127153 ... 2019-03-01

[8 rows x 12 columns]
<class 'pandas.core.frame.DataFrame'>
 psnb_total_spend_usd ... month_start_date
psnb_total_spend_usd 7717.582208 ... 2019-03-01
psb_spend 6931.919149 ... 2019-03-01
display_spend 4156.530690 ... 2019-03-01
tv_spend 6043.968095 ... 2019-03-01
affiliate_spend 4367.364092 ... 2019-03-01
psnb_total_clicks 7079.162143 ... 2019-03-01
psb_clicks 5656.904804 ... 2019-03-01
display_clicks 2459.564319 ... 2019-03-01

[8 rows x 12

In [6]:
final_df.head()

,psnb,psb,display,tv_radio,affiliate,email,discount,momentum,metric,date,country,total_revenue,exog_adj
0,0.221054,0.206530,0.058783,0.053801,0.072757,0.031782,0.089088,0.266204,new_total_bkgs,2019-03-01,US,15418318.72,0.265711
1,0.165685,0.176660,0.059674,0.035932,0.076640,0.092522,0.118724,0.274163,rpt_total_bkgs,2019-03-01,US,57575655.19,0.399688
2,0.218419,0.171898,0.056806,0.045470,0.059352,0.035564,0.040137,0.372353,order_count_new_bc,2019-03-01,US,195071.00,0.221812
3,0.090131,0.166733,0.054277,0.078289,0.033893,0.009274,0.153052,0.414351,order_count_new_cons,2019-03-01,US,43890.00,0.051996
4,0.245914,0.205318,0.060704,0.050757,0.071187,0.044164,0.008226,0.313730,order_count_new_other,2019-03-01,US,12478.00,0.200255


In [7]:
# final_irf_df_pivot.head()

# final_irf_df_pivot[final_irf_df_pivot['period'] == 19]

final_irf_df_1_pivot[final_irf_df_1_pivot['period'] == 19]

,affiliate_spend,display_clicks,display_spend,figure,month_start_date,period,psb_clicks,psb_spend,psnb_total_clicks,psnb_total_spend_usd,response,tv_spend
36,1948.441115,1533.705532,2803.939106,value,2019-03-01,19,1571.277944,2913.049561,2892.343360,3086.587228,psnb_total_spend_usd,2601.497648
37,2218.030754,1990.756415,2854.322415,value,2019-04-01,19,1034.277102,2879.601299,3367.183975,3716.231901,psnb_total_spend_usd,2205.986728
36,251.292032,-67.573124,388.180831,value,2019-03-01,19,68.918372,309.457259,38.504667,130.015348,psb_spend,179.420112
37,290.596486,6.307065,368.154649,value,2019-04-01,19,-13.179278,276.239298,12.988791,143.364371,psb_spend,83.881431
36,79.543278,202.959732,658.507462,value,2019-03-01,19,85.340344,21.076014,-9.074913,11.909972,display_spend,92.284302
37,147.961014,173.802288,522.052254,value,2019-04-01,19,156.147805,157.223497,271.937275,258.506053,display_spend,245.190864
36,-674.010217,-353.505174,367.719694,value,2019-03-01,19,-873.479035,-1020.612091,203.898668,-614.595714,tv_spend,-453.103182
37,-735.621459,54.276236,458.533714,value,2019-04-01,19,-1203.218332,-1141.768075,94.439635,-589.951284,tv_spend,-489.725531
36,84.286620,8.260761,133.140974,value,2019-03-01,19,57.427206,117.419617,93.650600,100.257200,affiliate_spend,98.222978
37,100.297849,50.569760,142.495510,value,2019-04-01,19,31.174613,116.030836,109.549585,128.756265,affiliate_spend,80.963030


In [8]:

# New Revenue Output

new_revenue_output = final_df[(final_df['metric'] == 'new_total_bkgs')]
new_revenue_output = new_revenue_output.rename(columns={"date": "month_start_date"})

new_revenue_output_spark = spark.createDataFrame(new_revenue_output)
new_revenue_output_spark.write.mode("overwrite").saveAsTable("rswaddle_US_new_revenue_test_output")

# Repeat Revenue Output

rpt_revenue_output = final_df[(final_df['metric'] == 'rpt_total_bkgs')]
rpt_revenue_output = rpt_revenue_output.rename(columns={"date": "month_start_date"})

rpt_revenue_output_spark = spark.createDataFrame(rpt_revenue_output)
rpt_revenue_output_spark.write.mode("overwrite").saveAsTable("rswaddle_US_rpt_revenue_test_output")

# New Order Output

new_order_output = final_df[~final_df.metric.isin(['new_total_bkgs', 'rpt_total_bkgs'])]
new_order_output = new_order_output.rename(columns={"metric": "product_category"})
new_order_output = new_order_output.rename(columns={"total_revenue": "new_orders"})
new_order_output = new_order_output.rename(columns={"date": "month_start_date"})

new_order_output = new_order_output.replace('order_count_new_bc', 'Business Cards')
new_order_output = new_order_output.replace('order_count_new_cons', 'Consumer')
new_order_output = new_order_output.replace('order_count_new_other', 'Other')
new_order_output = new_order_output.replace('order_count_new_plt', 'Packaging & Long Tail')
new_order_output = new_order_output.replace('order_count_new_ppag', 'PPAG')
new_order_output = new_order_output.replace('order_count_new_signage', 'Signage')
new_order_output = new_order_output.replace('order_count_new_mm', 'Marketing Materials')

new_order_output_spark = spark.createDataFrame(new_order_output)
new_order_output_spark.write.mode("overwrite").saveAsTable("rswaddle_US_new_order_test_output")

# New Revenue Log IRF

new_revenue_l_irf = final_irf_df_pivot[(final_irf_df_pivot['metric'] == 'new_total_bkgs_log')]
new_revenue_l_irf = new_revenue_l_irf.rename(columns={"metric_value": "l_new"})
new_revenue_l_irf = new_revenue_l_irf.rename(columns={"psnb_total_spend_usd": "l_psnb"})
new_revenue_l_irf = new_revenue_l_irf.rename(columns={"psb_spend": "l_psb"})
new_revenue_l_irf = new_revenue_l_irf.rename(columns={"display_spend": "l_disp"})
new_revenue_l_irf = new_revenue_l_irf.rename(columns={"tv_spend": "l_tv_radio"})
new_revenue_l_irf = new_revenue_l_irf.rename(columns={"affiliate_spend": "l_aff"})

new_revenue_l_irf_spark = spark.createDataFrame(new_revenue_l_irf)
new_revenue_l_irf_spark.write.mode("overwrite").saveAsTable("rswaddle_new_elasticity_table")

# Repeat Revenue Log IRF

rpt_revenue_l_irf = final_irf_df_pivot[(final_irf_df_pivot['metric'] == 'rpt_total_bkgs_log')]
rpt_revenue_l_irf = rpt_revenue_l_irf.rename(columns={"metric_value": "l_rpt"})
rpt_revenue_l_irf = rpt_revenue_l_irf.rename(columns={"psnb_total_spend_usd": "l_psnb"})
rpt_revenue_l_irf = rpt_revenue_l_irf.rename(columns={"psb_spend": "l_psb"})
rpt_revenue_l_irf = rpt_revenue_l_irf.rename(columns={"display_spend": "l_disp"})
rpt_revenue_l_irf = rpt_revenue_l_irf.rename(columns={"tv_spend": "l_tv_radio"})
rpt_revenue_l_irf = rpt_revenue_l_irf.rename(columns={"affiliate_spend": "l_aff"})

rpt_revenue_l_irf_spark = spark.createDataFrame(rpt_revenue_l_irf)
rpt_revenue_l_irf_spark.write.mode("overwrite").saveAsTable("rswaddle_rpt_elasticity_table")

# ST Dev New

new_std_df = final_std_df[(final_std_df['metric'] == 'new_total_bkgs_log')]
new_std_df = new_std_df.rename(columns={"metric_value": "l_new"})

new_std_df_spark = spark.createDataFrame(new_std_df)
new_std_df_spark.write.mode("overwrite").saveAsTable("rswaddle_new_std_table")

# ST Dev Rpt

rpt_std_df = final_std_df[(final_std_df['metric'] == 'rpt_total_bkgs_log')]
rpt_std_df = rpt_std_df.rename(columns={"metric_value": "l_rpt"})

rpt_std_df_spark = spark.createDataFrame(rpt_std_df)
rpt_std_df_spark.write.mode("overwrite").saveAsTable("rswaddle_rpt_std_table")

In [9]:
def mmm_elasticities_output(country, month_start_date, month_end_date):
  
  final_output_df = pd.DataFrame(columns=['locale', 'month_start_date', 'metric', 'psnb', 'psb', 'display', 'tv_radio', 'affiliate'])
  
  while month_start_date <= month_end_date :

    month_start_end = str(convert_to_var("select add_months('"+ month_start_date + "', 1) as date", "date"))

    for new_rpt in ["new","rpt"]:
      spark.sql("drop table if exists rswaddle_" + country + "_" + new_rpt + "_elasticity_table_pre")

      query = """
      create table  rswaddle_""" + country + """_""" + new_rpt + """_elasticity_table_pre
      as
      select distinct
      '""" + country + """' as locale
      , a.month_start_date
      , a.response
      , a.period
      , a.figure
      , case when abs(a.l_psnb) >= abs(b.l_psnb) then abs(a.l_psnb) else 0 end as l_psnb
      , case when abs(a.l_psb) >= abs(b.l_psb) then abs(a.l_psb) else 0 end as l_psb
      , case when abs(a.l_disp) >= abs(b.l_disp) then abs(a.l_disp) else 0 end as l_disp
      , case when abs(a.l_tv_radio) >= abs(b.l_tv_radio) then abs(a.l_tv_radio) else 0 end as l_tv_radio
      , case when abs(a.l_aff) >= abs(b.l_aff) then abs(a.l_aff) else 0 end as l_aff
      , case when abs(a.l_""" + new_rpt + """) >= abs(b.l_""" + new_rpt + """) then abs(a.l_""" + new_rpt + """) else 0 end as l_""" + new_rpt + """
      from
      rswaddle_""" + new_rpt + """_elasticity_table a
      left join
      (
      -- select * from rswaddle_""" + new_rpt + """_elasticity_table where figure = 'sd'
      select * from rswaddle_""" + new_rpt + """_elasticity_table where figure = 'value' and month_start_date = '""" + month_start_date + """'
      )
      b on 
      a.response = b.response
      and a.period = b.period
      and a.month_start_date = b.month_start_date
      where 
      a.figure = 'value'
      and a.month_start_date = '""" + month_start_date + """'
      order by
      a.response
      , a.period
      """
      spark.sql(query) 
#       print("rswaddle_" + country + "_" + new_rpt + "_elasticity_table_pre")


      spark.sql("drop table if exists rswaddle_" + country + "_" + new_rpt + "_elasticity_table_pre_2")

      query = """
      create table rswaddle_""" + country + """_""" + new_rpt + """_elasticity_table_pre_2
      as
      select distinct
      e.locale
      , e.month_start_date
      , e.response
      , ( 1.0 * e.l_psnb / sd.l_psnb ) as l_psnb
      , ( 1.0 * e.l_psb / sd.l_psb ) as l_psb
      , ( 1.0 * e.l_disp / sd.l_disp ) as l_disp
      , ( 1.0 * e.l_tv_radio / sd.l_tv_radio ) as l_tv_radio
      , ( 1.0 * e.l_aff / sd.l_aff ) as l_aff
      from
      (
      select distinct
      d.locale
      , d.month_start_date
      , d.response
      , sum(d.l_psnb) as l_psnb
      , sum(d.l_psb) as l_psb
      , sum(d.l_disp) as l_disp
      , sum(d.l_tv_radio) as l_tv_radio
      , sum(d.l_aff) as l_aff
      , sum(d.l_""" + new_rpt + """) as l_""" + new_rpt + """
      from
      (
      select distinct
      a.locale
      , a.month_start_date
      , a.response
      , a.period
      , a.figure
      , case  when b.l_psnb is null or c.l_psnb is null then a.l_psnb
              when ( b.l_psnb + c.l_psnb ) = 0 then 0
              else a.l_psnb
              end as l_psnb
      , case  when b.l_psb is null or c.l_psb is null then a.l_psb
              when ( b.l_psb + c.l_psb ) = 0 then 0
              else a.l_psb
              end as l_psb   
      , case  when b.l_disp is null or c.l_disp is null then a.l_disp
              when ( b.l_disp + c.l_disp ) = 0 then 0
              else a.l_disp
              end as l_disp
      , case  when b.l_tv_radio is null or c.l_tv_radio is null then a.l_tv_radio
              when ( b.l_tv_radio + c.l_tv_radio ) = 0 then 0
              else a.l_tv_radio
              end as l_tv_radio  
      , case  when b.l_aff is null or c.l_aff is null then a.l_aff
              when ( b.l_aff + c.l_aff ) = 0 then 0
              else a.l_aff
              end as l_aff     
      , case  when b.l_""" + new_rpt + """ is null or c.l_""" + new_rpt + """ is null then a.l_""" + new_rpt + """
              when ( b.l_""" + new_rpt + """ + c.l_""" + new_rpt + """ ) = 0 then 0
              else a.l_""" + new_rpt + """
              end as l_""" + new_rpt + """ 
      from
      rswaddle_""" + country + """_""" + new_rpt + """_elasticity_table_pre a
      left join rswaddle_""" + country + """_""" + new_rpt + """_elasticity_table_pre b 
      on a.response = b.response 
      and a.period = (b.period - 1)
      and a.month_start_date = b.month_start_date
      left join rswaddle_""" + country + """_""" + new_rpt + """_elasticity_table_pre c 
      on a.response = c.response 
      and a.period = (c.period - 2)
      and a.month_start_date = c.month_start_date
      where
      a.month_start_date = '""" + month_start_date + """'
      )
      d
      group by 
      d.locale
      , d.month_start_date
      , d.response
      )
      e
      left join rswaddle_""" + new_rpt + """_std_table sd on sd.country = e.locale and sd.month_start_date = e.month_start_date
      """
      spark.sql(query) 
#       print("rswaddle_" + country + "_" + new_rpt + "_elasticity_table_pre_2")

      spark.sql("drop table if exists rswaddle_" + country + "_" + new_rpt + "_elasticity_table_pre_3")

      query = """
      create table rswaddle_""" + country + """_""" + new_rpt + """_elasticity_table_pre_3
      as
      select distinct
      main.response
      , main.month_start_date
      , main.DISPLAY
      , main.TV_RADIO
      , main.PSNB
      , main.PSB
      , main.AFF
      , f.psb_spend
      , f.psnb_spend
      , f.display_spend
      , f.tv_radio_spend
      , f.aff_spend
      from
      (
      select distinct
      name as response
      , month_start_date
      , sum(case when response = 'display_spend' then value else 0 end) DISPLAY
      , sum(case when response = 'tv_spend' then value else 0 end) TV_RADIO
      , sum(case when response = 'psnb_total_spend_usd' then value else 0 end) PSNB
      , sum(case when response = 'psb_spend' then value else 0 end) PSB
      , sum(case when response = 'affiliate_spend' then value else 0 end) AFF
      from
      (
        select response, month_start_date, l_psnb value, 'PSNB' name
        from rswaddle_""" + country + """_""" + new_rpt + """_elasticity_table_pre_2
        union all
        select response, month_start_date, l_psb value, 'PSB' name
        from rswaddle_""" + country + """_""" + new_rpt + """_elasticity_table_pre_2
        union all
        select response, month_start_date, l_disp value, 'DISPLAY' name
        from rswaddle_""" + country + """_""" + new_rpt + """_elasticity_table_pre_2
        union all
        select response, month_start_date, l_tv_radio value, 'TV_RADIO' name
        from rswaddle_""" + country + """_""" + new_rpt + """_elasticity_table_pre_2
        union all
        select response, month_start_date, l_aff value, 'AFF' name
        from rswaddle_""" + country + """_""" + new_rpt + """_elasticity_table_pre_2
      ) src
      group by 
      name
      , month_start_date
      )
      main
      left join
      (
      (
      select distinct
      '""" + country + """' as country
      , '""" + month_start_date + """' as month_start_date
      , sum(psb_spend) as psb_spend
      , sum(psnb_total_spend_usd) as psnb_spend
      , sum(display_spend) as display_spend
      , sum(tv_spend + radio_spend) as tv_radio_spend
      , sum(affiliate_spend) as aff_spend
      from 
      rswaddle_""" + country + """_mmm_overall_data_clean
      where
      date >= '""" + month_start_date + """'
      and date < '""" + month_start_end + """'
      group by 
      '""" + country + """'
      , '""" + month_start_date + """'
      )
      b
      cross join
      (
      select distinct
      'PSB' as channel
      union all
      select distinct
      'PSNB'
      union all
      select distinct
      'TV_RADIO'
      union all
      select distinct
      'DISPLAY'
      union all
      select distinct
      'AFF'
      )
      c
      )
      f
      on main.response = f.channel
      and main.month_start_date = f.month_start_date
      """
      spark.sql(query) 
#       print("rswaddle_" + country + "_" + new_rpt + "_elasticity_table_pre_3")

    spark.sql("drop table if exists rswaddle_" + country + "_elasticity_table_pre_4")

    query = """
    create table rswaddle_""" + country + """_elasticity_table_pre_4
    as
    select distinct
    'new' as new_rpt
    , '""" + country + """' as locale
    , '""" + month_start_date + """' as month_start_date
    , sum(case when a.response = 'DISPLAY' then a.spend_contribution else 0 end) display
    , sum(case when a.response = 'TV_RADIO' then a.spend_contribution else 0 end) tv_radio
    , sum(case when a.response = 'PSNB' then a.spend_contribution else 0 end) psnb
    , sum(case when a.response = 'PSB' then a.spend_contribution else 0 end) psb
    , sum(case when a.response = 'AFF' then a.spend_contribution else 0 end) affiliate
    from
    (
    select distinct   
    response
    , case  when response = 'PSNB' then psnb_spend + (PSB * psb_spend) + (DISPLAY * display_spend) + (TV_RADIO * tv_radio_spend) + (AFF * aff_spend)
          when response = 'PSB' then psb_spend + (PSNB * psnb_spend) + (DISPLAY * display_spend) + (TV_RADIO * tv_radio_spend) + (AFF * aff_spend)
          when response = 'DISPLAY' then display_spend + (PSNB * psnb_spend) + (PSB * psb_spend) + (TV_RADIO * tv_radio_spend) + (AFF * aff_spend)
          when response = 'TV_RADIO' then tv_radio_spend + (PSNB * psnb_spend) + (DISPLAY * display_spend) + (PSB + psb_spend) + (AFF * aff_spend)
          when response = 'AFF' then aff_spend + (PSNB * psnb_spend) + (DISPLAY * display_spend) + (TV_RADIO * tv_radio_spend) + (PSB * psb_spend)
          else 0 
          end as spend_contribution
    from
    rswaddle_""" + country + """_new_elasticity_table_pre_3
    where 
    month_start_date = '""" + month_start_date + """'
    )
    a
    group by 
    'new'
    , '""" + country + """'
    , '""" + month_start_date + """'
    union all
    select distinct
    'rpt' as new_rpt
    , '""" + country + """' as locale
    , '""" + month_start_date + """' as month_start_date
    , sum(case when a.response = 'DISPLAY' then a.spend_contribution else 0 end) display
    , sum(case when a.response = 'TV_RADIO' then a.spend_contribution else 0 end) tv_radio
    , sum(case when a.response = 'PSNB' then a.spend_contribution else 0 end) psnb
    , sum(case when a.response = 'PSB' then a.spend_contribution else 0 end) psb
    , sum(case when a.response = 'AFF' then a.spend_contribution else 0 end) affiliate
    from
    (
    select distinct   
    response
    , case  when response = 'PSNB' then psnb_spend + (PSB * psb_spend) + (DISPLAY * display_spend) + (TV_RADIO * tv_radio_spend) + (AFF * aff_spend)
          when response = 'PSB' then psb_spend + (PSNB * psnb_spend) + (DISPLAY * display_spend) + (TV_RADIO * tv_radio_spend) + (AFF * aff_spend)
          when response = 'DISPLAY' then display_spend + (PSNB * psnb_spend) + (PSB * psb_spend) + (TV_RADIO * tv_radio_spend) + (AFF * aff_spend)
          when response = 'TV_RADIO' then tv_radio_spend + (PSNB * psnb_spend) + (DISPLAY * display_spend) + (PSB + psb_spend) + (AFF * aff_spend)
          when response = 'AFF' then aff_spend + (PSNB * psnb_spend) + (DISPLAY * display_spend) + (TV_RADIO * tv_radio_spend) + (PSB * psb_spend)
          else 0 
          end as spend_contribution
    from
    rswaddle_""" + country + """_rpt_elasticity_table_pre_3
    where
    month_start_date = '""" + month_start_date + """'
    )
    a
    group by 
    'rpt'
    , '""" + country + """'
    , '""" + month_start_date + """'
    """
    spark.sql(query) 
#     print("rswaddle_" + country + "_elasticity_table_pre_4")

    spark.sql("drop table if exists rswaddle_" + country + "_elasticities_output_1")

    query = """
    create table rswaddle_""" + country + """_elasticities_output_1
    as
    select distinct
    a.locale
    , a.month_start_date
    , 'auc_total' as metric
    , sum(a.l_psnb) as psnb
    , sum(a.l_psb) as psb
    , sum(a.l_disp) as display
    , sum(a.l_tv_radio) as tv_radio
    , sum(a.l_aff) as affiliate
    from
    (
    select distinct
    n.locale
    , n.month_start_date
    , 'new' as new_rpt
    , ( n.l_psnb * new_pct ) as l_psnb
    , ( n.l_psb * new_pct ) as l_psb
    , ( n.l_disp * new_pct ) as l_disp
    , ( n.l_tv_radio * new_pct ) as l_tv_radio
    , ( n.l_aff * new_pct ) as l_aff
    from 
    rswaddle_""" + country + """_new_elasticity_table_pre_2 n
    left join 
    (
    select distinct
    '""" + country + """' as locale
    , '""" + month_start_date + """' as month_start_date
    , (sum(new_total_bkgs) / sum(new_total_bkgs + rpt_total_bkgs) ) as new_pct
    from
    rswaddle_""" + country + """_mmm_overall_data_clean
    where
    date >= '""" + month_start_date + """'
    and date < '""" + month_start_end + """'
    group by 
    '""" + country + """'
    , '""" + month_start_date + """'
    )
    o
    on n.locale = o.locale
    and n.month_start_date = o.month_start_date
    where 
    response = 'metric_value'  
    and n.month_start_date = '""" + month_start_date + """'
    union all
    select distinct
    n.locale
    , n.month_start_date
    , 'rpt' as new_rpt
    , ( n.l_psnb * rpt_pct ) as l_psnb
    , ( n.l_psb * rpt_pct ) as l_psb
    , ( n.l_disp * rpt_pct ) as l_disp
    , ( n.l_tv_radio * rpt_pct ) as l_tv_radio
    , ( n.l_aff * rpt_pct ) as l_aff
    from 
    rswaddle_""" + country + """_rpt_elasticity_table_pre_2 n
    left join 
    (
    select distinct
    '""" + country + """' as locale
    , '""" + month_start_date + """' as month_start_date
    , sum(rpt_total_bkgs) / sum(new_total_bkgs + rpt_total_bkgs) as rpt_pct
    from
    rswaddle_""" + country + """_mmm_overall_data_clean
    where
    date >= '""" + month_start_date + """'
    and date < '""" + month_start_end + """'
    group by 
    '""" + country + """'
    , '""" + month_start_date + """'
    )
    o
    on n.locale = o.locale
    and n.month_start_date = o.month_start_date
    where 
    response = 'metric_value' 
    and n.month_start_date = '""" + month_start_date + """'
    )
    a
    group by 
    a.locale
    , a.month_start_date
    , 'auc_total'
    union all
    select distinct
    locale
    , month_start_date
    , 'auc_new' as metric
    , l_psnb as psnb
    , l_psb as psb
    , l_disp as display
    , l_tv_radio as tv_radio
    , l_aff as affiliate
    from
    rswaddle_""" + country + """_new_elasticity_table_pre_2
    where
    response = 'metric_value'
    and month_start_date = '""" + month_start_date + """'
    union all
    select distinct
    locale
    , month_start_date
    , 'auc_rpt' as metric
    , l_psnb as psnb
    , l_psb as psb
    , l_disp as display
    , l_tv_radio as tv_radio
    , l_aff as affiliate
    from
    rswaddle_""" + country + """_rpt_elasticity_table_pre_2
    where
    response = 'metric_value'
    and month_start_date = '""" + month_start_date + """'
    """
    spark.sql(query) 
#     print("rswaddle_" + country + "_elasticities_output_1")


    spark.sql("drop table if exists rswaddle_" + country + "_elasticities_output_2")

    query = """
    create table rswaddle_""" + country + """_elasticities_output_2
    as
    select distinct * from rswaddle_""" + country + """_elasticities_output_1
    union all
    select distinct
    o.locale
    , o.month_start_date
    , 'total_bookings' as metric
    , o.psnb * b.monthly_bkgs as psnb
    , o.psb * b.monthly_bkgs as psb
    , o.display * b.monthly_bkgs as display
    , o.tv_radio * b.monthly_bkgs as tv_radio
    , o.affiliate * b.monthly_bkgs as affiliate
    from
    rswaddle_""" + country + """_elasticities_output_1 o
    left join 
    (
    select distinct
    '""" + country + """' as locale
    , '""" + month_start_date + """' as month_start_date
    , sum(new_total_bkgs + rpt_total_bkgs) as monthly_bkgs
    from
    rswaddle_""" + country + """_mmm_overall_data_clean
    where
    date >= '""" + month_start_date + """'
    and date < '""" + month_start_end + """'
    group by 
    '""" + country + """'
    , '""" + month_start_date + """'
    )
    b on 
    o.locale = b.locale
    and o.month_start_date = b.month_start_date
    where 
    metric = 'auc_total'
    and o.month_start_date = '""" + month_start_date + """'
    union all
    select distinct
    o.locale
    , o.month_start_date
    , 'total_agp' as metric
    , o.psnb * b.monthly_bkgs * gp.agp_ratio as psnb
    , o.psb * b.monthly_bkgs * gp.agp_ratio as psb
    , o.display * b.monthly_bkgs * gp.agp_ratio as display
    , o.tv_radio * b.monthly_bkgs * gp.agp_ratio as tv_radio
    , o.affiliate * b.monthly_bkgs * gp.agp_ratio as affiliate
    from
    rswaddle_""" + country + """_elasticities_output_1 o
    left join 
    (
    select distinct
    '""" + country + """' as locale
    , '""" + month_start_date + """' as month_start_date
    , sum(new_total_bkgs + rpt_total_bkgs) as monthly_bkgs
    from
    rswaddle_""" + country + """_mmm_overall_data_clean
    where
    date >= '""" + month_start_date + """'
    and date < '""" + month_start_end + """'
    group by 
    '""" + country + """'
    , '""" + month_start_date + """'
    )
    b on 
    o.locale = b.locale
    and o.month_start_date = b.month_start_date
    left join
    (
    select distinct
    website_country as locale
    , '""" + month_start_date + """' as month_start_date
    , sum(finance_adjusted_gp) / sum(mse_bkgs) as agp_ratio
    from
    rswaddle_MSE_credit_and_extraCOGS_proxies
    where
    website_country = '""" + country + """'
    and month_start_date = '""" + month_start_date + """'
    group by 
    website_country
    )
    gp
    on gp.locale = o.locale
    and gp.month_start_date = o.month_start_date
    where 
    metric = 'auc_total'
    and o.month_start_date = '""" + month_start_date + """'
    union all
    select distinct
    '""" + country + """' as locale
    , '""" + month_start_date + """' as month_start_date
    , 'total_spend' as metric
    , m.psnb
    , m.psb
    , m.display
    , m.tv_radio
    , m.affiliate
    from
    (
    select distinct
    '""" + month_start_date + """' as month_start_date
    , case when n.display > r.display then n.display else r.display end as display
    , case when n.tv_radio > r.tv_radio then n.tv_radio else r.tv_radio end as tv_radio
    , case when n.psnb > r.psnb then n.psnb else r.psnb end as psnb
    , case when n.psb > r.psb then n.psb else r.psb end as psb
    , case when n.affiliate > r.affiliate then n.affiliate else r.affiliate end as affiliate
    from
    rswaddle_""" + country + """_elasticity_table_pre_4 n
    left join 
    (select * from rswaddle_""" + country + """_elasticity_table_pre_4 where new_rpt = 'rpt') r on n.locale = r.locale and n.month_start_date = n.month_start_date
    where 
    n.new_rpt = 'new'
    and n.month_start_date = '""" + month_start_date + """'
    )
    m
    """
    spark.sql(query) 
#     print("rswaddle_" + country + "_elasticities_output_2")

    spark.sql("drop table if exists rswaddle_" + country + "_elasticities_output_3")

    query = """
    create table rswaddle_""" + country + """_elasticities_output_3
    as
    select distinct * from rswaddle_""" + country + """_elasticities_output_2
    union all
    select distinct
    '""" + country + """' as locale
    , a.month_start_date
    , 'spend_shock_agp_roi' as metric
    , a.psnb / s.psnb -1 as psnb
    , a.psb / s.psb -1 as psb
    , a.display / s.display -1 as display
    , a.tv_radio / s.tv_radio -1 as tv_radio
    , a.affiliate / s.affiliate -1 as affiliate
    from
    rswaddle_""" + country + """_elasticities_output_2 a
    left join (select * from rswaddle_""" + country + """_elasticities_output_2 where metric = 'total_spend' and month_start_date = '""" + month_start_date + """' ) s 
    on s.locale = a.locale
    and s.month_start_date = a.month_start_date
    where
    a.metric = 'total_agp'
    and a.month_start_date = '""" + month_start_date + """'
    union all
    select distinct
    a.locale
    , a.month_start_date
    , 'relative_elasticity' as metric
    , a.psnb / t.total_auc as psnb
    , a.psb / t.total_auc as psb
    , a.display / t.total_auc as display
    , a.tv_radio / t.total_auc as tv_radio
    , a.affiliate / t.total_auc as affiliate
    from
    rswaddle_""" + country + """_elasticities_output_2 a
    left join 
    (
    select distinct 
    locale
    , month_start_date
    , ( psnb + psb + display + tv_radio + affiliate ) as total_auc
    from
    rswaddle_""" + country + """_elasticities_output_2
    where 
    metric = 'auc_total'
    and month_start_date = '""" + month_start_date + """'
    )
    t
    on 
    a.locale = t.locale
    and a.month_start_date = t.month_start_date
    where 
    a.metric = 'auc_total'
    and a.month_start_date = '""" + month_start_date + """'
    """
    spark.sql(query) 
    print("rswaddle_" + country + "_elasticities_output_3")
    
    print("Date " + month_start_date + " complete")
    
    month_start_date = str(convert_to_var("select add_months('"+ month_start_date + "', 1) as date", "date"))
    
    output_df = spark.sql("SELECT * FROM rswaddle_" + country + "_elasticities_output_3").toPandas()
    
#     print(output_df)
    
    final_output_df = final_output_df.append(output_df, ignore_index=True)
    
    
  return final_output_df

In [10]:
final_output_df = mmm_elasticities_output('US','2019-03-01', '2019-04-01')

rswaddle_US_elasticities_output_3
Date 2019-03-01 complete
rswaddle_US_elasticities_output_3
Date 2019-04-01 complete

In [11]:
final_output_df

,locale,month_start_date,metric,psnb,psb,display,tv_radio,affiliate
0,US,2019-03-01,relative_elasticity,2.789684e-01,3.032720e-01,2.360673e-01,1.531204e-02,1.663802e-01
1,US,2019-03-01,spend_shock_agp_roi,1.788634e-02,2.840804e-01,1.033022e-01,-6.333845e-01,1.885707e-01
2,US,2019-03-01,total_bookings,7.108002e+07,7.727246e+07,6.014898e+07,3.901445e+06,4.239300e+07
3,US,2019-03-01,total_spend,4.001125e+07,3.447994e+07,3.123689e+07,6.097454e+06,2.043634e+07
4,US,2019-03-01,auc_total,9.737792e-01,1.058614e+00,8.240267e-01,5.344887e-02,5.807740e-01
5,US,2019-03-01,total_agp,4.072691e+07,4.427501e+07,3.446372e+07,2.235422e+06,2.429003e+07
6,US,2019-03-01,auc_rpt,9.076633e-01,1.034262e+00,8.226531e-01,4.481511e-02,5.513495e-01
7,US,2019-03-01,auc_new,1.220672e+00,1.149552e+00,8.291559e-01,8.568936e-02,6.906521e-01
8,US,2019-04-01,spend_shock_agp_roi,3.354588e-02,4.229904e-01,2.650801e-01,-3.874677e-01,2.612634e-01
9,US,2019-04-01,relative_elasticity,2.441264e-01,3.199848e-01,2.359002e-01,2.578578e-02,1.742028e-01


In [12]:
final_output_df_spark = spark.createDataFrame(final_output_df)
final_output_df_spark.write.mode("overwrite").saveAsTable("rswaddle_us_elasticity_output")

In [13]:
final_irf_df_1_pivot_spark = spark.createDataFrame(final_irf_df_1_pivot)
final_irf_df_1_pivot_spark.write.mode("overwrite").saveAsTable("rswaddle_us_channel_interactions")

In [14]:
%sql
refresh table rswaddle_us_channel_interactions ; 

-- select * from rswaddle_elasticity_output ; 
select * from rswaddle_us_channel_interactions ; 

affiliate_spend,display_clicks,display_spend,figure,month_start_date,period,psb_clicks,psb_spend,psnb_total_clicks,psnb_total_spend_usd,response,tv_spend
9692.448487921305,4320.475171551433,6685.574028341674,value,2019-03-01,1,16403.283753879263,16546.44149674785,19649.236388424,20875.55288569837,psnb_total_spend_usd,3176.5004265729112
10201.513338287767,4824.883424747174,6934.871345879548,value,2019-04-01,1,15534.376179982448,16598.021935144698,19514.595576445667,20708.306747604285,psnb_total_spend_usd,3248.904413908954
8186.042804493721,3663.5866272540493,5975.143089215196,value,2019-03-01,2,13977.120500377474,14170.589408015447,17134.394205767603,18110.69073960443,psnb_total_spend_usd,4928.171540173072
8550.446170794323,4281.334834391618,6297.706384984214,value,2019-04-01,2,13122.417773185296,14163.311587256336,17005.8013898687,18019.890503843606,psnb_total_spend_usd,4867.9990425536225
7262.968090749972,3153.1467627351003,5294.688258811918,value,2019-03-01,3,11832.416967303221,12229.209931044361,14645.15047535767,15599.456046800202,psnb_total_spend_usd,5921.590553571968
2502.7334578474065,1809.78095875482,3276.1971597655397,value,2019-03-01,16,2384.526015956064,3827.1015560563183,3639.5645274941376,3951.9037280666485,psnb_total_spend_usd,3453.6649758324306
2799.609718061387,2325.418266527206,3459.4454536870976,value,2019-04-01,16,1756.9199109301437,3746.292567157073,4079.153382532189,4560.946218491127,psnb_total_spend_usd,3037.0641471899726
2303.613390800676,1714.2420598817457,3120.28144201592,value,2019-03-01,17,2080.638578922676,3496.550647204789,3355.0638049170816,3626.4904761758503,psnb_total_spend_usd,3145.7408552087245
2591.260010755231,2210.139927638694,3256.081468173756,value,2019-04-01,17,1483.2543276742845,3430.235928695313,3808.2180680978126,4244.474423836645,psnb_total_spend_usd,2735.2247624928264
2119.0704057757475,1621.9775608931313,2962.3236579457366,value,2019-03-01,18,1810.3627095390073,3192.185762869392,3107.3045034556467,3339.354885814675,psnb_total_spend_usd,2861.6890018704444


In [15]:

###############################
#####  DATA ARCHITECTURE  ##### 
###############################

## INPUT TABLES ##

# commons.shopper_ltv
# rswaddle_uk_new_revenue_test_output
# rswaddle_uk_rpt_revenue_test_output
# rswaddle_uk_new_order_test_output
# rswaddle_UK_mmm_overall_data_clean
# rswaddle_mse_spend_data_table (TEMPORARY SOLUTION)
# rswaddle_MSE_credit_and_extraCOGS_proxies
# rswaddle_mmm_channels


## OUTPUT TABLES ##

# rswaddle_mmm_avg_ltv_agp_test
# rswaddle_mmm_pdt_channel_output_test
# rswaddle_mmm_channel_output_test
# rswaddle_mmm_overview_output_test


In [16]:
%sql
select * from rswaddle_US_rpt_revenue_test_output limit 10 ; 

psnb,psb,display,tv_radio,affiliate,email,discount,momentum,metric,month_start_date,country,total_revenue,exog_adj
0.16568491319559248,0.17665953006772878,0.05967444260955503,0.03593193897549146,0.07664026294192271,0.09252185146947493,0.11872361822352138,0.27416344251671315,rpt_total_bkgs,2019-03-01,US,5.757565518999999E7,0.399688271667863
0.1491928151288433,0.1787566161982608,0.0470755060800526,0.03716356982549175,0.07796034732928686,0.10044574951916375,0.12120757593241484,0.288197819986486,rpt_total_bkgs,2019-04-01,US,5.9596107660000004E7,0.41764134394264707


In [17]:
def mmm_channel_output(country):

  spark.sql("drop table if exists rswaddle_" + country + "_mmm_channel_output_test")
  spark.sql("refresh table rswaddle_MSE_credit_and_extraCOGS_proxies")
  spark.sql("refresh table rswaddle_" + country + "_rpt_revenue_test_output")
  spark.sql("refresh table rswaddle_" + country + "_new_revenue_test_output ")
  spark.sql("refresh table rswaddle_" + country + "_mmm_overall_data_clean")
  
  query = """
  
  create table rswaddle_""" + country + """_mmm_channel_output_test
  as
  select distinct
  a.month_start_date
  , a.channel
  , case when a.channel = 'PaidSearchNonBranded' then (1.0 * gp.agp_true_up * new.psnb * new.total_revenue)
         when a.channel = 'PaidSearchBranded' then (1.0 * gp.agp_true_up * new.psb * new.total_revenue)
         when a.channel = 'Display' then (1.0 * gp.agp_true_up * new.display * new.total_revenue)
         when a.channel = 'TV_Radio' then (1.0 * gp.agp_true_up * new.tv_radio * new.total_revenue)
         when a.channel = 'Affiliate' then (1.0 * gp.agp_true_up * new.affiliate * new.total_revenue)
         when a.channel = 'Discount' then (1.0 * gp.agp_true_up * new.discount * new.total_revenue)
         when a.channel = 'Email' then (1.0 * gp.agp_true_up * new.email * new.total_revenue)
         when a.channel = 'Momentum' then (1.0 * gp.agp_true_up * new.momentum * new.total_revenue)
         else 0
         end as new_agp_contribution
  , case when a.channel = 'PaidSearchNonBranded' then (1.0 * gp.agp_true_up * rpt.psnb * rpt.total_revenue)
         when a.channel = 'PaidSearchBranded' then (1.0 * gp.agp_true_up * rpt.psb * rpt.total_revenue)
         when a.channel = 'Display' then (1.0 * gp.agp_true_up * rpt.display * rpt.total_revenue)
         when a.channel = 'TV_Radio' then (1.0 * gp.agp_true_up * rpt.tv_radio * rpt.total_revenue)
         when a.channel = 'Affiliate' then (1.0 * gp.agp_true_up * rpt.affiliate * rpt.total_revenue)
         when a.channel = 'Discount' then (1.0 * gp.agp_true_up * rpt.discount * rpt.total_revenue)
         when a.channel = 'Email' then (1.0 * gp.agp_true_up * rpt.email * rpt.total_revenue)
         when a.channel = 'Momentum' then (1.0 * gp.agp_true_up * rpt.momentum * rpt.total_revenue)
         else 0
         end as rpt_agp_contribution
  , case when a.channel = 'PaidSearchNonBranded' then (1.0 * new.psnb * new.total_revenue)
         when a.channel = 'PaidSearchBranded' then (1.0 * new.psb * new.total_revenue)
         when a.channel = 'Display' then (1.0 * new.display * new.total_revenue)
         when a.channel = 'TV_Radio' then (1.0 * new.tv_radio * new.total_revenue)
         when a.channel = 'Affiliate' then (1.0 * new.affiliate * new.total_revenue)
         when a.channel = 'Discount' then (1.0 * new.discount * new.total_revenue)
         when a.channel = 'Email' then (1.0 * new.email * new.total_revenue)
         when a.channel = 'Momentum' then (1.0 * new.momentum * new.total_revenue)
         else 0
         end as new_bookings_contribution
  , case when a.channel = 'PaidSearchNonBranded' then (1.0 * rpt.psnb * rpt.total_revenue)
         when a.channel = 'PaidSearchBranded' then (1.0 * rpt.psb * rpt.total_revenue)
         when a.channel = 'Display' then (1.0 * rpt.display * rpt.total_revenue)
         when a.channel = 'TV_Radio' then (1.0 * rpt.tv_radio * rpt.total_revenue)
         when a.channel = 'Affiliate' then (1.0 * rpt.affiliate * rpt.total_revenue)
         when a.channel = 'Discount' then (1.0 * rpt.discount * rpt.total_revenue)
         when a.channel = 'Email' then (1.0 * rpt.email * rpt.total_revenue)
         when a.channel = 'Momentum' then (1.0 * rpt.momentum * rpt.total_revenue)
         else 0
         end as rpt_bookings_contribution       
  , case when a.channel = 'PaidSearchNonBranded' then spend.psnb_spend
         when a.channel = 'PaidSearchBranded' then spend.psb_spend
         when a.channel = 'Display' then spend.display_spend
         when a.channel = 'TV_Radio' then spend.tv_radio_spend
         when a.channel = 'Affiliate' then spend.aff_spend
         when a.channel = 'Discount' then 0
         when a.channel = 'Email' then 0
         when a.channel = 'Momentum' then 0
         else 0
         end as spend_usd    
  , case when a.channel = 'PaidSearchNonBranded' then (new.psnb)
         when a.channel = 'PaidSearchBranded' then (new.psb)
         when a.channel = 'Display' then (new.display)
         when a.channel = 'TV_Radio' then (new.tv_radio)
         when a.channel = 'Affiliate' then (new.affiliate)
         when a.channel = 'Discount' then (new.discount)
         when a.channel = 'Email' then (new.email)
         when a.channel = 'Momentum' then (new.momentum)
         else 0
         end as new_contribution_pct     
  , case when a.channel = 'PaidSearchNonBranded' then (rpt.psnb)
         when a.channel = 'PaidSearchBranded' then (rpt.psb)
         when a.channel = 'Display' then (rpt.display)
         when a.channel = 'TV_Radio' then (rpt.tv_radio)
         when a.channel = 'Affiliate' then (rpt.affiliate)
         when a.channel = 'Discount' then (rpt.discount)
         when a.channel = 'Email' then (rpt.email)
         when a.channel = 'Momentum' then (rpt.momentum)
         else 0
         end as rpt_contribution_pct       
  from
  (
  select distinct
  r.month_start_date
  , c.channel
  from
  rswaddle_""" + country + """_rpt_revenue_test_output r
  cross join
  (
  select distinct
  'PaidSearchNonBranded' as channel
  union all 
  select distinct
  'PaidSearchBranded' as channel
  union all
  select distinct
  'Affiliate' as channel
  union all
  select distinct
  'TV_Radio' as channel
  union all
  select distinct
  'Display' as channel
  union all
  select distinct
  'Discount' as channel
  union all
  select distinct
  'Email' as channel
  union all
  select distinct
  'Momentum' as channel
  )
  c
  )
  a 
  left join 
  (
  select distinct
  ot.country as locale
  , ot.month_start_date
  , ot.total_revenue
  , ( (1.0 - ot.exog_adj) * (case when null_spend.psnb_spend < 50 then 0 else ot.psnb end ) ) as psnb 
  , ( (1.0 - ot.exog_adj) * (case when null_spend.psb_spend < 50 then 0 else ot.psb end ) ) as psb 
  , ( (1.0 - ot.exog_adj) * (case when null_spend.display_spend < 50 then 0 else ot.display end ) ) as display 
  , ( (1.0 - ot.exog_adj) * (case when null_spend.tv_radio_spend < 50 then 0 else ot.tv_radio end ) ) as tv_radio
  , ( (1.0 - ot.exog_adj) * ot.affiliate ) as affiliate
  , ( (1.0 - ot.exog_adj) * ot.discount ) as discount
  , ( (1.0 - ot.exog_adj) * ot.email ) as email
  , 1.0 - ( ( (1.0 - ot.exog_adj) * (case when null_spend.psnb_spend < 50 then 0 else ot.psnb end ) ) + ( (1.0 - ot.exog_adj) * (case when null_spend.psb_spend < 50 then 0 else ot.psb end ) ) + ( (1.0 - ot.exog_adj) * (case when null_spend.display_spend < 50 then 0 else ot.display end ) ) + ( ( 1.0 - ot.exog_adj) * (case when null_spend.tv_radio_spend < 50 then 0 else ot.tv_radio end ) ) + ( ( 1.0 - ot.exog_adj) * ot.affiliate ) + ( ( 1.0 - ot.exog_adj) * ot.discount ) + ( ( 1.0 - ot.exog_adj) * ot.email ) ) as momentum
  from 
  rswaddle_""" + country + """_rpt_revenue_test_output ot
  left join
  (
  select distinct
  dt.month_start_date
  , sum(psb_spend) as psb_spend
  , sum(psnb_total_spend_usd) as psnb_spend
  , sum(display_spend) as display_spend
  , sum(tv_spend + radio_spend) as tv_radio_spend
  from 
  rswaddle_""" + country + """_mmm_overall_data_clean o
  left join vdw.dim_date dt on dt.date = o.date
  group by 
  dt.month_start_date
  )
  null_spend
  on null_spend.month_start_date = ot.month_start_date
  )
  rpt 
  on rpt.month_start_date = a.month_start_date
  left join 
  (
  select distinct
  ot2.country as locale
  , ot2.month_start_date
  , ot2.total_revenue
  , ( (1.0 - ot2.exog_adj) * (case when null_spend_2.psnb_spend < 50 then 0 else ot2.psnb end )  ) as psnb 
  , ( (1.0 - ot2.exog_adj) * (case when null_spend_2.psb_spend < 50 then 0 else ot2.psb end ) ) as psb 
  , ( (1.0 - ot2.exog_adj) * (case when null_spend_2.display_spend < 50 then 0 else ot2.display end ) ) as display 
  , ( (1.0 - ot2.exog_adj) * (case when null_spend_2.tv_radio_spend < 50 then 0 else ot2.tv_radio end ) ) as tv_radio
  , ( (1.0 - ot2.exog_adj) * ot2.affiliate ) as affiliate
  , ( (1.0 - ot2.exog_adj) * ot2.discount ) as discount
  , ( (1.0 - ot2.exog_adj) * ot2.email ) as email
  , 1.0 - ( ( (1.0 - ot2.exog_adj) * (case when null_spend_2.psnb_spend < 50 then 0 else ot2.psnb end ) ) + ( (1.0 - ot2.exog_adj) * (case when null_spend_2.psb_spend < 50 then 0 else ot2.psb end ) ) + ( (1.0 - ot2.exog_adj) * (case when null_spend_2.display_spend < 50 then 0 else ot2.display end ) ) + ( ( 1.0 - ot2.exog_adj) * (case when null_spend_2.tv_radio_spend < 50 then 0 else ot2.tv_radio end ) ) + ( ( 1.0 - ot2.exog_adj) * ot2.affiliate ) + ( ( 1.0 - ot2.exog_adj) * ot2.discount ) + ( ( 1.0 - ot2.exog_adj) * ot2.email ) ) as momentum
  from 
  rswaddle_""" + country + """_new_revenue_test_output ot2
  left join
  (
  select distinct
  dt.month_start_date
  , sum(psb_spend) as psb_spend
  , sum(psnb_total_spend_usd) as psnb_spend
  , sum(display_spend) as display_spend
  , sum(tv_spend + radio_spend) as tv_radio_spend
  from 
  rswaddle_""" + country + """_mmm_overall_data_clean o
  left join vdw.dim_date dt on dt.date = o.date
  group by 
  dt.month_start_date
  )
  null_spend_2
  on null_spend_2.month_start_date = ot2.month_start_date
  )
  new 
  on new.month_start_date = a.month_start_date
  left join
  (
  select distinct
  month_start_date
  , sum(mse_bkgs) mse_bkgs
  , sum(finance_adjusted_gp) as finance_adjusted_gp
  , ( 1.0 * sum(finance_adjusted_gp) / sum(mse_bkgs) ) as agp_true_up
  from
  rswaddle_MSE_credit_and_extraCOGS_proxies
  where 
  country_group in ( '""" + country + """' )
  group by 
  month_start_date
  )
  gp
  on gp.month_start_date = a.month_start_date
  left join
  (
  select distinct
  a.month_start_date
  , a.psnb_spend
  , a.psb_spend
  , a.display_spend
  , a.tv_radio_spend
  , a.affiliate_spend as aff_spend
  from
  (
  select distinct
  dt.month_start_date
  , sum(psnb_total_spend_usd) as psnb_spend
  , sum(psb_spend) as psb_spend
  , sum(display_spend) as display_spend
  , sum(tv_spend + radio_spend) as tv_radio_spend
  , sum(affiliate_spend) as affiliate_spend
  from 
  rswaddle_""" + country + """_mmm_overall_data_clean c
  left join vdw.dim_date dt on dt.date = c.date
  group by 
  dt.month_start_date
  )
  a
  )
  spend
  on spend.month_start_date = a.month_start_date
  where
  rpt.locale in ( '""" + country + """' )
  """
  spark.sql(query) 
  print("rswaddle_" + country + "_mmm_channel_output_test")

In [18]:
def mmm_avg_ltv_agp(country):

  spark.sql("refresh table commons.shopper_ltv ")
  spark.sql("refresh table rswaddle_MSE_credit_and_extraCOGS_proxies ")
  spark.sql("drop table if exists rswaddle_" + country + "_mmm_avg_ltv_agp_test")

  query = """
  
  create table rswaddle_""" + country + """_mmm_avg_ltv_agp_test
  as
  select distinct
  ltv.fo_month_start_date
  , case  when ltv.fo_pi_mktg_category in ('Business Cards','Consumer','Marketing Materials','Signage') then fo_pi_mktg_category
          when ltv.fo_pi_mktg_category in ('PPAG Clothing & Bags','PPAG Hard Goods') then 'PPAG'
          when ltv.fo_pi_mktg_category in ('Packaging & Stationery','Packaging & Long Tail') then 'Packaging & Long Tail'
          else 'Other'
          end as fo_pi_mktg_category
  , count(distinct ltv.shopper_key) as count_shopper_key
  , t.Adj_GP_factor *  ( ( sum(ltv.fo_gm_budget_usd) ) / count(distinct ltv.shopper_key) ) as avg_fo_agp
  , t.Adj_GP_factor * ( ( sum(ltv.m3_gm_budget_usd_ltv) - sum(ltv.fo_gm_budget_usd) ) / count(distinct ltv.shopper_key) ) as avg_m1_m3_rpt_agp
  , t.Adj_GP_factor * ( ( sum(ltv.m6_gm_budget_usd_ltv) - sum(ltv.m3_gm_budget_usd_ltv) ) / count(distinct ltv.shopper_key) ) as avg_m4_m6_rpt_agp
  , t.Adj_GP_factor * ( ( sum(ltv.y1_gm_budget_usd_ltv) - sum(ltv.m6_gm_budget_usd_ltv) ) / count(distinct ltv.shopper_key) ) as avg_m7_m12_rpt_agp
  , t.Adj_GP_factor * ( ( sum(ltv.y2_gm_budget_usd_ltv) - sum(ltv.y1_gm_budget_usd_ltv) ) / count(distinct ltv.shopper_key) ) as avg_m13_m24_rpt_agp
  , t.Adj_GP_factor * ( ( sum(ltv.y3_gm_budget_usd_ltv) - sum(ltv.y2_gm_budget_usd_ltv) ) / count(distinct ltv.shopper_key) ) as avg_m25_m36_rpt_agp
  , t.Adj_GP_factor * ( ( sum(ltv.y4_gm_budget_usd_ltv) - sum(ltv.y3_gm_budget_usd_ltv) ) / count(distinct ltv.shopper_key) ) as avg_m37_m48_rpt_agp
  , t.Adj_GP_factor * ( ( sum(ltv.y5_gm_budget_usd_ltv) - sum(ltv.y4_gm_budget_usd_ltv) ) / count(distinct ltv.shopper_key) ) as avg_m49_m60_rpt_agp
  from commons.shopper_ltv ltv
  left join (select * from rswaddle_MSE_credit_and_extraCOGS_proxies where country_group in ( '""" + country + """' ) ) t on t.month_start_date = ltv.fo_month_start_date
  where 
  ltv.fo_country_group in ( '""" + country + """' )
  and ltv.fo_month_start_date>='2017-01-01' 
  --and ltv.fo_month_start_date<'2020-01-01'
  group by 
  ltv.fo_month_start_date
  , case  when ltv.fo_pi_mktg_category in ('Business Cards','Consumer','Marketing Materials','Signage') then fo_pi_mktg_category
          when ltv.fo_pi_mktg_category in ('PPAG Clothing & Bags','PPAG Hard Goods') then 'PPAG'
           when ltv.fo_pi_mktg_category in ('Packaging & Stationery','Packaging & Long Tail') then 'Packaging & Long Tail'
          else 'Other'
          end
  , t.Adj_GP_factor
  """
  spark.sql(query) 
  print("rswaddle_" + country + "_mmm_avg_ltv_agp_test")

In [19]:
def mmm_pdt_channel_output(country):

  spark.sql("drop table if exists rswaddle_" + country + "_mmm_pdt_channel_output_test")
  spark.sql("refresh table rswaddle_" + country + "_new_order_test_output")
  spark.sql("refresh table rswaddle_" + country + "_mmm_avg_ltv_agp_test")
            
  query = """
  
  create table rswaddle_""" + country + """_mmm_pdt_channel_output_test
  as
  select distinct
  m.month_start_date
  , m.product_category
  , m.channel
  , case when m.channel = 'PaidSearchNonBranded' then ( 1.0 * o.psnb * a.paid_avg_fo_agp )
         when m.channel = 'PaidSearchBranded' then ( 1.0 * o.psb * a.paid_avg_fo_agp )
         when m.channel = 'Display' then ( 1.0 * o.display * a.paid_avg_fo_agp )
         when m.channel = 'Affiliate' then ( 1.0 * o.affiliate * a.paid_avg_fo_agp )
         when m.channel = 'TV_Radio' then ( 1.0 * o.tv_radio * a.paid_avg_fo_agp )
         else 0 
         end as paid_avg_fo_agp
  , case when m.channel = 'PaidSearchNonBranded' then ( 1.0 * o.psnb * a.baseline_avg_m1_m3_rpt_agp )
         when m.channel = 'PaidSearchBranded' then ( 1.0 * o.psb * a.baseline_avg_m1_m3_rpt_agp )
         when m.channel = 'Display' then ( 1.0 * o.display * a.baseline_avg_m1_m3_rpt_agp )
         when m.channel = 'Affiliate' then ( 1.0 * o.affiliate * a.baseline_avg_m1_m3_rpt_agp )
         when m.channel = 'TV_Radio' then ( 1.0 * o.tv_radio * a.baseline_avg_m1_m3_rpt_agp )
         else 0 
         end as baseline_avg_m1_m3_rpt_agp
  , case when m.channel = 'PaidSearchNonBranded' then ( 1.0 * o.psnb * a.baseline_avg_m4_m6_rpt_agp )
         when m.channel = 'PaidSearchBranded' then ( 1.0 * o.psb * a.baseline_avg_m4_m6_rpt_agp )
         when m.channel = 'Display' then ( 1.0 * o.display * a.baseline_avg_m4_m6_rpt_agp )
         when m.channel = 'Affiliate' then ( 1.0 * o.affiliate * a.baseline_avg_m4_m6_rpt_agp )
         when m.channel = 'TV_Radio' then ( 1.0 * o.tv_radio * a.baseline_avg_m4_m6_rpt_agp )
         else 0 
         end as baseline_avg_m4_m6_rpt_agp
  , case when m.channel = 'PaidSearchNonBranded' then ( 1.0 * o.psnb * a.baseline_avg_m7_m12_rpt_agp )
         when m.channel = 'PaidSearchBranded' then ( 1.0 * o.psb * a.baseline_avg_m7_m12_rpt_agp )
         when m.channel = 'Display' then ( 1.0 * o.display * a.baseline_avg_m7_m12_rpt_agp )
         when m.channel = 'Affiliate' then ( 1.0 * o.affiliate * a.baseline_avg_m7_m12_rpt_agp )
         when m.channel = 'TV_Radio' then ( 1.0 * o.tv_radio * a.baseline_avg_m7_m12_rpt_agp )
         else 0 
         end as baseline_avg_m7_m12_rpt_agp
  , case when m.channel = 'PaidSearchNonBranded' then ( 1.0 * o.psnb * a.baseline_avg_m13_m24_rpt_agp )
         when m.channel = 'PaidSearchBranded' then ( 1.0 * o.psb * a.baseline_avg_m13_m24_rpt_agp )
         when m.channel = 'Display' then ( 1.0 * o.display * a.baseline_avg_m13_m24_rpt_agp )
         when m.channel = 'Affiliate' then ( 1.0 * o.affiliate * a.baseline_avg_m13_m24_rpt_agp )
         when m.channel = 'TV_Radio' then ( 1.0 * o.tv_radio * a.baseline_avg_m13_m24_rpt_agp )
         else 0 
         end as baseline_avg_m13_m24_rpt_agp 
  , case when m.channel = 'PaidSearchNonBranded' then ( 1.0 * o.psnb * a.baseline_avg_m25_m36_rpt_agp )
         when m.channel = 'PaidSearchBranded' then ( 1.0 * o.psb * a.baseline_avg_m25_m36_rpt_agp )
         when m.channel = 'Display' then ( 1.0 * o.display * a.baseline_avg_m25_m36_rpt_agp )
         when m.channel = 'Affiliate' then ( 1.0 * o.affiliate * a.baseline_avg_m25_m36_rpt_agp )
         when m.channel = 'TV_Radio' then ( 1.0 * o.tv_radio * a.baseline_avg_m25_m36_rpt_agp )
         else 0 
         end as baseline_avg_m25_m36_rpt_agp 
  , case when m.channel = 'PaidSearchNonBranded' then ( 1.0 * o.psnb * a.baseline_avg_m37_m48_rpt_agp )
         when m.channel = 'PaidSearchBranded' then ( 1.0 * o.psb * a.baseline_avg_m37_m48_rpt_agp )
         when m.channel = 'Display' then ( 1.0 * o.display * a.baseline_avg_m37_m48_rpt_agp )
         when m.channel = 'Affiliate' then ( 1.0 * o.affiliate * a.baseline_avg_m37_m48_rpt_agp )
         when m.channel = 'TV_Radio' then ( 1.0 * o.tv_radio * a.baseline_avg_m37_m48_rpt_agp )
         else 0 
         end as baseline_avg_m37_m48_rpt_agp 
  , case when m.channel = 'PaidSearchNonBranded' then ( 1.0 * o.psnb * a.baseline_avg_m49_m60_rpt_agp )
         when m.channel = 'PaidSearchBranded' then ( 1.0 * o.psb * a.baseline_avg_m49_m60_rpt_agp )
         when m.channel = 'Display' then ( 1.0 * o.display * a.baseline_avg_m49_m60_rpt_agp )
         when m.channel = 'Affiliate' then ( 1.0 * o.affiliate * a.baseline_avg_m49_m60_rpt_agp )
         when m.channel = 'TV_Radio' then ( 1.0 * o.tv_radio * a.baseline_avg_m49_m60_rpt_agp )
         else 0 
         end as baseline_avg_m49_m60_rpt_agp        
  from
  (
  select distinct
  o.month_start_date
  , o.product_category
  , c.dim_channel as channel
  from
  rswaddle_""" + country + """_new_order_test_output o
  cross join rswaddle_mmm_channels c
  )
  m
  left join 
  (
  select distinct
  o.country as locale
  , o.month_start_date
  , o.product_category
  , ( ( o.new_orders * ( 1 - o.exog_adj ) ) * (case when null_spend.psnb_spend < 50 then 0 else o.psnb end ) ) as psnb
  , ( ( o.new_orders * ( 1 - o.exog_adj ) ) * (case when null_spend.psb_spend < 50 then 0 else o.psb end ) ) as psb
  , ( ( o.new_orders * ( 1 - o.exog_adj ) ) * (case when null_spend.display_spend < 50 then 0 else o.display end ) ) as display
  , ( ( o.new_orders * ( 1 - o.exog_adj ) ) * (case when null_spend.tv_radio_spend < 50 then 0 else o.tv_radio end ) ) as tv_radio
  , ( ( o.new_orders * ( 1 - o.exog_adj ) ) * o.affiliate ) as affiliate
  , ( ( o.new_orders * ( 1 - o.exog_adj ) ) * o.discount ) as discount
  , ( ( o.new_orders * ( 1 - o.exog_adj ) ) * o.email ) as email
  --, ( ( ( o.new_orders * ( 1 - o.exog_adj ) ) * o.momentum ) + ( o.new_orders * o.exog_adj ) ) as momentum
  , ( o.new_orders * ( 1.0 - ( ( (1.0 - o.exog_adj) * (case when null_spend.psnb_spend < 50 then 0 else o.psnb end ) ) + ( (1.0 - o.exog_adj) * (case when null_spend.psb_spend < 50 then 0 else o.psb end ) ) + ( (1.0 - o.exog_adj) * (case when null_spend.display_spend < 50 then 0 else o.display end ) ) + ( ( 1.0 - o.exog_adj) * (case when null_spend.tv_radio_spend < 50 then 0 else o.tv_radio end ) ) + ( ( 1.0 - o.exog_adj) * o.affiliate ) + ( ( 1.0 - o.exog_adj) * o.discount ) + ( ( 1.0 - o.exog_adj) * o.email ) ) ) ) as momentum
  from
  rswaddle_""" + country + """_new_order_test_output o 
  left join
  (
  select distinct
  dt.month_start_date
  , sum(psb_spend) as psb_spend
  , sum(psnb_total_spend_usd) as psnb_spend
  , sum(display_spend) as display_spend
  , sum(tv_spend + radio_spend) as tv_radio_spend
  from 
  rswaddle_""" + country + """_mmm_overall_data_clean o
  left join vdw.dim_date dt on dt.date = o.date
  group by 
  dt.month_start_date
  )
  null_spend
  on null_spend.month_start_date = o.month_start_date
  )
  o 
  on o.month_start_date = m.month_start_date 
  and o.product_category = m.product_category
  left join
  (
  select distinct
  ltv.fo_month_start_date as month_start_date
  , ltv.fo_pi_mktg_category as mktg_category
  -- , (1.0 * new.new_paid_contribution * ltv.avg_fo_agp ) as paid_avg_fo_agp
  , (1.0 * ltv.avg_fo_agp ) as paid_avg_fo_agp
  , (1.0 * rpt.rpt_baseline_contribution * ltv.avg_m1_m3_rpt_agp ) as baseline_avg_m1_m3_rpt_agp
  , (1.0 * rpt.rpt_baseline_contribution * ltv.avg_m4_m6_rpt_agp ) as baseline_avg_m4_m6_rpt_agp
  , (1.0 * rpt.rpt_baseline_contribution * ltv.avg_m7_m12_rpt_agp ) as baseline_avg_m7_m12_rpt_agp
  , (1.0 * rpt.rpt_baseline_contribution * ltv.avg_m13_m24_rpt_agp ) as baseline_avg_m13_m24_rpt_agp
  , (1.0 * rpt.rpt_baseline_contribution * ltv.avg_m25_m36_rpt_agp ) as baseline_avg_m25_m36_rpt_agp
  , (1.0 * rpt.rpt_baseline_contribution * ltv.avg_m37_m48_rpt_agp ) as baseline_avg_m37_m48_rpt_agp
  , (1.0 * rpt.rpt_baseline_contribution * ltv.avg_m49_m60_rpt_agp ) as baseline_avg_m49_m60_rpt_agp
  from
  rswaddle_""" + country + """_mmm_avg_ltv_agp_test ltv
  -- left join
  -- (
  -- select distinct
  -- dt.month_start_date
  -- , ( 1 - ( new.discount + new.email + new.momentum ) ) as new_paid_contribution
  -- from
  -- rswaddle_""" + country + """_new_revenue_test_output new
  -- left join vdw.dim_date dt on dt.date = new.month_date
  -- )
  -- new 
  -- on new.month_start_date = ltv.fo_month_start_date
  left join
  (
  select distinct
  dt.month_start_date
  , ( rpt.discount + rpt.email + rpt.momentum ) as rpt_baseline_contribution
  from
  (
  select distinct
  o2.country as locale
  , o2.month_start_date
  , ( (1.0 - o2.exog_adj) * (case when null_spend_2.psnb_spend < 50 then 0 else o2.psnb end ) ) as psnb 
  , ( (1.0 - o2.exog_adj) * (case when null_spend_2.psb_spend < 50 then 0 else o2.psb end ) ) as psb 
  , ( (1.0 - o2.exog_adj) * (case when null_spend_2.display_spend < 50 then 0 else o2.display end ) ) as display 
  , ( (1.0 - o2.exog_adj) * (case when null_spend_2.tv_radio_spend < 50 then 0 else o2.tv_radio end ) ) as tv_radio
  , ( (1.0 - o2.exog_adj) * o2.affiliate ) as affiliate
  , ( (1.0 - o2.exog_adj) * o2.discount ) as discount
  , ( (1.0 - o2.exog_adj) * o2.email ) as email
  , 1.0 - ( ( (1.0 - o2.exog_adj) * (case when null_spend_2.psnb_spend < 50 then 0 else o2.psnb end ) ) + ( (1.0 - o2.exog_adj) * (case when null_spend_2.psb_spend < 50 then 0 else o2.psb end ) ) + ( (1.0 - o2.exog_adj) * (case when null_spend_2.display_spend < 50 then 0 else o2.display end ) ) + ( ( 1.0 - o2.exog_adj) * (case when null_spend_2.tv_radio_spend < 50 then 0 else o2.tv_radio end ) ) + ( ( 1.0 - o2.exog_adj) * o2.affiliate ) + ( ( 1.0 - o2.exog_adj) * o2.discount ) + ( ( 1.0 - o2.exog_adj) * o2.email ) ) as momentum
  from 
  rswaddle_""" + country + """_rpt_revenue_test_output o2
  left join
  (
  select distinct
  dt.month_start_date
  , sum(psb_spend) as psb_spend
  , sum(psnb_total_spend_usd) as psnb_spend
  , sum(display_spend) as display_spend
  , sum(tv_spend + radio_spend) as tv_radio_spend
  from 
  rswaddle_""" + country + """_mmm_overall_data_clean o
  left join vdw.dim_date dt on dt.date = o.date
  group by 
  dt.month_start_date
  )
  null_spend_2
  on null_spend_2.month_start_date = o2.month_start_date
  )
  rpt
  left join vdw.dim_date dt on dt.date = rpt.month_start_date
  )
  rpt 
  on rpt.month_start_date = ltv.fo_month_start_date
  )
  a
  on a.month_start_date = m.month_start_date
  and a.mktg_category = m.product_category
  """
  spark.sql(query) 
  print("rswaddle_" + country + "_mmm_pdt_channel_output_test")

In [20]:
def mmm_overview_output(country):
  
#   mmm_channel_output(country)
#   mmm_avg_ltv_agp(country)
#   mmm_pdt_channel_output(country)

  spark.sql("drop table if exists rswaddle_" + country + "_mmm_overview_output_pre_test")
  spark.sql("refresh table rswaddle_MSE_credit_and_extraCOGS_proxies")

  query = """
  
  create table rswaddle_""" + country + """_mmm_overview_output_pre_test
  as
  select distinct
  a.month_start_date
  , a.channel
  , a.spend_usd
  , b.paid_avg_fo_agp as paid_total_fo_agp_comparison
  , a.new_agp_contribution as paid_total_fo_agp
  , a.rpt_agp_contribution as paid_total_rpt_agp
  , a.new_bookings_contribution as paid_total_fo_bkgs
  , a.rpt_bookings_contribution as paid_total_rpt_bkgs
  , p.total_agp_new as p3x_new_agp
  , p.total_agp_rpt as p3x_rpt_agp
  , p.total_bkgs_new as p3x_new_bkgs
  , p.total_bkgs_rpt as p3x_rpt_bkgs
  , b.baseline_avg_m1_m3_rpt_agp
  , b.baseline_avg_m4_m6_rpt_agp
  , b.baseline_avg_m7_m12_rpt_agp
  , b.baseline_avg_m13_m24_rpt_agp
  , b.baseline_avg_m25_m36_rpt_agp
  , b.baseline_avg_m37_m48_rpt_agp
  , b.baseline_avg_m49_m60_rpt_agp
  , b.baseline_avg_m1_m3_rpt_agp / ( b.paid_avg_fo_agp / a.new_agp_contribution ) as baseline_avg_m1_m3_rpt_agp_adj
  , b.baseline_avg_m4_m6_rpt_agp / ( b.paid_avg_fo_agp / a.new_agp_contribution ) as baseline_avg_m4_m6_rpt_agp_adj
  , b.baseline_avg_m7_m12_rpt_agp / ( b.paid_avg_fo_agp / a.new_agp_contribution ) as baseline_avg_m7_m12_rpt_agp_adj
  , b.baseline_avg_m13_m24_rpt_agp / ( b.paid_avg_fo_agp / a.new_agp_contribution ) as baseline_avg_m13_m24_rpt_agp_adj
  , b.baseline_avg_m25_m36_rpt_agp / ( b.paid_avg_fo_agp / a.new_agp_contribution ) as baseline_avg_m25_m36_rpt_agp_adj
  , b.baseline_avg_m37_m48_rpt_agp / ( b.paid_avg_fo_agp / a.new_agp_contribution ) as baseline_avg_m37_m48_rpt_agp_adj
  , b.baseline_avg_m49_m60_rpt_agp / ( b.paid_avg_fo_agp / a.new_agp_contribution ) as baseline_avg_m49_m60_rpt_agp_adj
  , ( b.paid_avg_fo_agp / a.new_agp_contribution ) as fo_agp_ratio
  from
  rswaddle_""" + country + """_mmm_channel_output_test a
  left join
  (
  select distinct
  month_start_date
  , channel
  , sum(paid_avg_fo_agp) as paid_avg_fo_agp
  , sum(baseline_avg_m1_m3_rpt_agp) as baseline_avg_m1_m3_rpt_agp
  , sum(baseline_avg_m4_m6_rpt_agp) as baseline_avg_m4_m6_rpt_agp
  , sum(baseline_avg_m7_m12_rpt_agp) as baseline_avg_m7_m12_rpt_agp
  , sum(baseline_avg_m13_m24_rpt_agp) as baseline_avg_m13_m24_rpt_agp
  , sum(baseline_avg_m25_m36_rpt_agp) as baseline_avg_m25_m36_rpt_agp
  , sum(baseline_avg_m37_m48_rpt_agp) as baseline_avg_m37_m48_rpt_agp
  , sum(baseline_avg_m49_m60_rpt_agp) as baseline_avg_m49_m60_rpt_agp
  from
  rswaddle_""" + country + """_mmm_pdt_channel_output_test
  group by 
  month_start_date
  , channel
  ) 
  b 
  on b.month_start_date = a.month_start_date
  and b.channel = a.channel
  left join
  (
  select distinct
  dt.month_start_date
  , d.channel
  , sum(d.total_bkgs_new * (ad.finance_adjusted_gp / ad.mse_bkgs)) as total_agp_new
  , sum(d.total_bkgs_rpt* (ad.finance_adjusted_gp / ad.mse_bkgs)) as total_agp_rpt
  , sum(d.total_bkgs_new ) as total_bkgs_new
  , sum(d.total_bkgs_rpt ) as total_bkgs_rpt
  from
  rswaddle_channel_bkgs_""" + country + """_mmm_data d
  left join vdw.dim_date dt on dt.date = d.order_date
  left join (select * from rswaddle_MSE_credit_and_extraCOGS_proxies where country_group in ( '""" + country + """' ) ) ad on ad.month_start_date = dt.month_start_date
  group by 
  dt.month_start_date
  , d.channel
  ) p
  on p.month_start_date = a.month_start_date and p.channel = a.channel
  where
  a.channel not in ('Email','Discount','Momentum')
  """
  spark.sql(query) 
  
  spark.sql("drop table if exists rswaddle_" + country + "_mmm_overview_output_test")

  query = """
  
  create table rswaddle_""" + country + """_mmm_overview_output_test
  as
  select distinct
  pre.month_start_date
  , pre.channel
  , pre.spend_usd
  , pre.paid_total_fo_agp_comparison
  , pre.paid_total_fo_agp
  , pre.paid_total_rpt_agp
  , pre.paid_total_fo_bkgs
  , pre.paid_total_rpt_bkgs
  , pre.p3x_new_agp
  , pre.p3x_rpt_agp
  , pre.p3x_new_bkgs
  , pre.p3x_rpt_bkgs
  , pre.baseline_avg_m1_m3_rpt_agp
  , pre.baseline_avg_m4_m6_rpt_agp
  , pre.baseline_avg_m7_m12_rpt_agp
  , pre.baseline_avg_m13_m24_rpt_agp
  , pre.baseline_avg_m25_m36_rpt_agp
  , pre.baseline_avg_m37_m48_rpt_agp
  , pre.baseline_avg_m49_m60_rpt_agp
  , roi.roi_fq
  , roi.roi_y1
  , roi.roi_y2
  , roi.roi_y3
  , roi.roi_y4
  , roi.roi_y5
  , roi.payback_months
  , roi2.payback_months as payback_months_adj
  , ( pre.paid_total_fo_agp_comparison / pre.paid_total_fo_agp ) as fo_agp_ratio
  --, roi.negative_years
  --, roi.last_negative_year
  --, roi.first_positive_year
  --, roi.delta
  --, roi.months_in_year_pb
  from
  rswaddle_""" + country + """_mmm_overview_output_pre_test pre
  left join
  (
  select distinct
  a.month_start_date
  , a.channel
  , a.roi_fq
  , a.roi_y1
  , a.roi_y2
  , a.roi_y3
  , a.roi_y4
  , a.roi_y5
  , a.negative_years
  , a.last_negative_year
  , a.first_positive_year
  , a.delta
  , case when a.last_negative_year <0 then -a.last_negative_year / a.delta * 12 else 0 end as months_in_year_pb
  , case when a.negative_years = 5 then 60
         else round(( (12 * a.negative_years) + case when a.last_negative_year <0 then -a.last_negative_year / a.delta * 12 else 0 end),0)
         end as payback_months
  from
  (
  select distinct
  month_start_date
  , channel
  , roi_fq
  , roi_y1
  , roi_y2
  , roi_y3
  , roi_y4
  , roi_y5
  , sum( case when roi_y1 < 0 then 1 else 0 end + case when roi_y2 < 0 then 1 else 0 end + case when roi_y3 < 0 then 1 else 0 end + case when roi_y4 < 0 then 1 else 0 end + case when roi_y5 < 0 then 1 else 0 end ) as negative_years
  , case  when roi_y5 < 0 then 0
          when roi_y4 < 0 then roi_y4
          when roi_y3 < 0 then roi_y3
          when roi_y2 < 0 then roi_y2
          when roi_y1 < 0 then roi_y1
          when roi_fq < 0 then roi_fq
          else roi_fq
          end as last_negative_year
  , case  when roi_fq > 0  then roi_y1
          when roi_y1 > 0 then roi_y2
          when roi_y2 > 0 then roi_y2
          when roi_y3 > 0 then roi_y3
          when roi_y4 > 0 then roi_y4
          when roi_y5 > 0 then roi_y5
          else 0
          end as first_positive_year
  , ( case  when roi_fq > 0  then roi_y1
          when roi_y1 > 0 then roi_y2
          when roi_y2 > 0 then roi_y2
          when roi_y3 > 0 then roi_y3
          when roi_y4 > 0 then roi_y4
          when roi_y5 > 0 then roi_y5
          else 0
          end ) 
          -
          (case  when roi_y5 < 0 then 0
          when roi_y4 < 0 then roi_y4
          when roi_y3 < 0 then roi_y3
          when roi_y2 < 0 then roi_y2
          when roi_y1 < 0 then roi_y1
          when roi_fq < 0 then roi_fq
          else roi_fq
          end ) as delta
  from
  (
  select distinct
  month_start_date
  , channel
  , sum( - spend_usd + paid_total_fo_agp + paid_total_rpt_agp) as roi_fq
  , sum( - spend_usd + paid_total_fo_agp + paid_total_rpt_agp + baseline_avg_m1_m3_rpt_agp + baseline_avg_m4_m6_rpt_agp + baseline_avg_m7_m12_rpt_agp ) as roi_y1
  , sum( - spend_usd + paid_total_fo_agp + paid_total_rpt_agp + baseline_avg_m1_m3_rpt_agp + baseline_avg_m4_m6_rpt_agp + baseline_avg_m7_m12_rpt_agp + baseline_avg_m13_m24_rpt_agp ) as roi_y2
  , sum( - spend_usd + paid_total_fo_agp + paid_total_rpt_agp + baseline_avg_m1_m3_rpt_agp + baseline_avg_m4_m6_rpt_agp + baseline_avg_m7_m12_rpt_agp + baseline_avg_m13_m24_rpt_agp + baseline_avg_m25_m36_rpt_agp ) as roi_y3
  , sum( - spend_usd + paid_total_fo_agp + paid_total_rpt_agp + baseline_avg_m1_m3_rpt_agp + baseline_avg_m4_m6_rpt_agp + baseline_avg_m7_m12_rpt_agp + baseline_avg_m13_m24_rpt_agp + baseline_avg_m25_m36_rpt_agp + baseline_avg_m37_m48_rpt_agp ) as roi_y4
  , sum( - spend_usd + paid_total_fo_agp + paid_total_rpt_agp + baseline_avg_m1_m3_rpt_agp + baseline_avg_m4_m6_rpt_agp + baseline_avg_m7_m12_rpt_agp + baseline_avg_m13_m24_rpt_agp + baseline_avg_m25_m36_rpt_agp + baseline_avg_m37_m48_rpt_agp + baseline_avg_m49_m60_rpt_agp ) as roi_y5
  from
  rswaddle_""" + country + """_mmm_overview_output_pre_test
  group by 
  month_start_date
  , channel
  )
  group by 
  month_start_date
  , channel
  , roi_fq
  , roi_y1
  , roi_y2
  , roi_y3
  , roi_y4
  , roi_y5
  )
  a
  )
  roi
  on pre.month_start_date = roi.month_start_date
  and pre.channel = roi.channel
  
  left join
  (
  select distinct
  a.month_start_date
  , a.channel
  , a.roi_fq
  , a.roi_y1
  , a.roi_y2
  , a.roi_y3
  , a.roi_y4
  , a.roi_y5
  , a.negative_years
  , a.last_negative_year
  , a.first_positive_year
  , a.delta
  , case when a.last_negative_year <0 then -a.last_negative_year / a.delta * 12 else 0 end as months_in_year_pb
  , case when a.negative_years = 5 then 60
         else round(( (12 * a.negative_years) + case when a.last_negative_year <0 then -a.last_negative_year / a.delta * 12 else 0 end),0)
         end as payback_months
  from
  (
  select distinct
  month_start_date
  , channel
  , roi_fq
  , roi_y1
  , roi_y2
  , roi_y3
  , roi_y4
  , roi_y5
  , sum( case when roi_y1 < 0 then 1 else 0 end + case when roi_y2 < 0 then 1 else 0 end + case when roi_y3 < 0 then 1 else 0 end + case when roi_y4 < 0 then 1 else 0 end + case when roi_y5 < 0 then 1 else 0 end ) as negative_years
  , case  when roi_y5 < 0 then 0
          when roi_y4 < 0 then roi_y4
          when roi_y3 < 0 then roi_y3
          when roi_y2 < 0 then roi_y2
          when roi_y1 < 0 then roi_y1
          when roi_fq < 0 then roi_fq
          else roi_fq
          end as last_negative_year
  , case  when roi_fq > 0  then roi_y1
          when roi_y1 > 0 then roi_y2
          when roi_y2 > 0 then roi_y2
          when roi_y3 > 0 then roi_y3
          when roi_y4 > 0 then roi_y4
          when roi_y5 > 0 then roi_y5
          else 0
          end as first_positive_year
  , ( case  when roi_fq > 0  then roi_y1
          when roi_y1 > 0 then roi_y2
          when roi_y2 > 0 then roi_y2
          when roi_y3 > 0 then roi_y3
          when roi_y4 > 0 then roi_y4
          when roi_y5 > 0 then roi_y5
          else 0
          end ) 
          -
          (case  when roi_y5 < 0 then 0
          when roi_y4 < 0 then roi_y4
          when roi_y3 < 0 then roi_y3
          when roi_y2 < 0 then roi_y2
          when roi_y1 < 0 then roi_y1
          when roi_fq < 0 then roi_fq
          else roi_fq
          end ) as delta
  from
  (
  select distinct
  month_start_date
  , channel
  , sum( - spend_usd + paid_total_fo_agp + paid_total_rpt_agp) as roi_fq
  , sum( - spend_usd + paid_total_fo_agp + paid_total_rpt_agp + baseline_avg_m1_m3_rpt_agp_adj + baseline_avg_m4_m6_rpt_agp_adj + baseline_avg_m7_m12_rpt_agp_adj ) as roi_y1
  , sum( - spend_usd + paid_total_fo_agp + paid_total_rpt_agp + baseline_avg_m1_m3_rpt_agp_adj + baseline_avg_m4_m6_rpt_agp_adj + baseline_avg_m7_m12_rpt_agp_adj + baseline_avg_m13_m24_rpt_agp_adj ) as roi_y2
  , sum( - spend_usd + paid_total_fo_agp + paid_total_rpt_agp + baseline_avg_m1_m3_rpt_agp_adj + baseline_avg_m4_m6_rpt_agp_adj + baseline_avg_m7_m12_rpt_agp_adj + baseline_avg_m13_m24_rpt_agp_adj + baseline_avg_m25_m36_rpt_agp_adj ) as roi_y3
  , sum( - spend_usd + paid_total_fo_agp + paid_total_rpt_agp + baseline_avg_m1_m3_rpt_agp_adj + baseline_avg_m4_m6_rpt_agp_adj + baseline_avg_m7_m12_rpt_agp_adj + baseline_avg_m13_m24_rpt_agp_adj + baseline_avg_m25_m36_rpt_agp_adj + baseline_avg_m37_m48_rpt_agp_adj ) as roi_y4
  , sum( - spend_usd + paid_total_fo_agp + paid_total_rpt_agp + baseline_avg_m1_m3_rpt_agp_adj + baseline_avg_m4_m6_rpt_agp_adj + baseline_avg_m7_m12_rpt_agp_adj + baseline_avg_m13_m24_rpt_agp_adj + baseline_avg_m25_m36_rpt_agp_adj + baseline_avg_m37_m48_rpt_agp_adj + baseline_avg_m49_m60_rpt_agp_adj ) as roi_y5
  from
  rswaddle_""" + country + """_mmm_overview_output_pre_test
  group by 
  month_start_date
  , channel
  )
  group by 
  month_start_date
  , channel
  , roi_fq
  , roi_y1
  , roi_y2
  , roi_y3
  , roi_y4
  , roi_y5
  )
  a
  )
  roi2
  on pre.month_start_date = roi2.month_start_date
  and pre.channel = roi2.channel
  
  """
  spark.sql(query) 
  print("rswaddle_" + country + "_mmm_overview_output_test")

In [21]:
mmm_overview_output('US')

rswaddle_US_mmm_overview_output_test

In [22]:
%sql
refresh table rswaddle_US_mmm_overview_output_test ; 

-- select * from rswaddle_US_mmm_overview_output_test ;
-- select * from rswaddle_US_mmm_channel_output_test ; 
-- select * from rswaddle_US_mmm_pdt_channel_output_test ; 
select * from rswaddle_US_mmm_avg_ltv_agp_test ; 

fo_month_start_date,fo_pi_mktg_category,count_shopper_key,avg_fo_agp,avg_m1_m3_rpt_agp,avg_m4_m6_rpt_agp,avg_m7_m12_rpt_agp,avg_m13_m24_rpt_agp,avg_m25_m36_rpt_agp,avg_m37_m48_rpt_agp,avg_m49_m60_rpt_agp
2017-09-01,Other,9279,5.335765758654993,51.51171864480032,11.467736554444235,19.112005206795235,31.273571244965407,23.13740570331678,19.243003137089623,18.026324217214217
2018-07-01,Marketing Materials,14579,49.12853758054159,26.245441513272773,12.703595857189384,23.094961995653893,32.74300909733514,28.874777505617402,26.349536649805213,21.299488665993817
2020-04-01,Business Cards,64976,26.884539698401063,8.901169275432059,5.04909890161556,9.251160733285838,15.210227897496315,12.566176742064924,11.33359619769777,10.300449338253078
2017-06-01,Business Cards,163897,20.17856788920201,7.40856550666168,4.272888591531198,7.346417868098778,11.601072266733814,8.888629167134875,8.922125761274629,8.276284862883188
2018-08-01,PPAG,8264,24.08743479359582,11.652809049177245,3.929279742933001,6.817649497065535,8.379186263053484,7.530602818040762,6.945824116964535,5.706693177495591
2018-06-01,PPAG,7571,25.883215145588693,10.932631930857209,3.4204153082936055,6.364056365727401,9.008393185116208,7.379817456507659,6.9387249135834,5.5902667379507776
2017-05-01,Business Cards,191747,18.540164663972117,6.845289263101624,4.12780792458474,7.092215197784255,11.490333452691488,9.175259110976102,8.576713734841565,7.937851138304252
2017-08-01,Marketing Materials,15316,52.23237839050256,26.137216828831225,13.763385808720026,26.083404079485597,41.471064764213224,30.11446294047732,27.227390959264312,25.186721233697273
2018-12-01,Packaging & Long Tail,39821,19.898330300757937,4.885123397428892,3.0068414905957535,5.742188967434036,8.6319248200215,7.963020533904516,7.5998496416037655,7.168507653306145
2019-02-01,Business Cards,161708,21.031776748747262,8.033665022747467,4.180689964658804,7.443234046998852,12.083173801469533,10.285927463160483,9.33749411664609,8.53372978309435


In [23]:

# rswaddle_US_mmm_overview_output_test
# rswaddle_US_mmm_pdt_channel_output_test
# rswaddle_US_mmm_avg_ltv_agp_test
# rswaddle_US_mmm_channel_output_test


In [24]:
dbutils.library.installPyPI("requests")

Out[15]: True

In [25]:
try:
  #Install, import and configure our channel_dna library
  dbutils.library.install('dbfs:/FileStore/jars/channel_dna/master/channel_dna_master.egg')
  import channel_dna 
  channel_dna.spark_config.set_spark_configuration(sqlContext=sqlContext, dbutils=dbutils, spark=spark)
except:
  raise

In [26]:
help(channel_dna.dal.snowflake.table_from_df) 

Help on method table_from_df in module channel_dna.data_access_layer:

table_from_df(df, database, schema, name, mode) method of channel_dna.data_access_layer.Snowflake instance
 Persist dataframe in a table in Snowflake.
 
 If the table doesn't exist it will be created automatically.
 
 You can Append the dataframe to an existing table (if the table schema
 is the same) or you can Overwrite the whole table.
 
 Parameters
 ----------
 df : spark dataframe
 Dataframe containing the information we want to persist in
 our table.
 database : str
 Name of the schema in snowflake
 schema : str
 Name of the schema in Snowflake.
 name : str
 Name of the table in Snowflake.
 mode : str
 Mode to persist the data.
 * Append: add the data in the dataframe to an existing table,
 table and datafrane must have the same columns
 in the same order.
 * Overwrite: drop the table and recreate it using
 the data passed in the dataframe.
 
 Raises
 ------
 Exception
 If execution fails.

In [27]:
query = """
select *
from rswaddle_US_mmm_overview_output_test
"""
df = channel_dna.dal.databricks.execute_reader(query=query)
display(df)

month_start_date,channel,spend_usd,paid_total_fo_agp_comparison,paid_total_fo_agp,paid_total_rpt_agp,paid_total_fo_bkgs,paid_total_rpt_bkgs,p3x_new_agp,p3x_rpt_agp,p3x_new_bkgs,p3x_rpt_bkgs,baseline_avg_m1_m3_rpt_agp,baseline_avg_m4_m6_rpt_agp,baseline_avg_m7_m12_rpt_agp,baseline_avg_m13_m24_rpt_agp,baseline_avg_m25_m36_rpt_agp,baseline_avg_m37_m48_rpt_agp,baseline_avg_m49_m60_rpt_agp,roi_fq,roi_y1,roi_y2,roi_y3,roi_y4,roi_y5,payback_months,payback_months_adj,fo_agp_ratio
2019-04-01,PaidSearchNonBranded,3856979.24,1176035.1997234395,1233650.1497948235,2998087.2959844633,2130610.591292558,5177931.966778874,1383096.8749132995,1314522.966271485,2388716.8100000005,2270284.2900000005,352505.1116681304,168047.39525740698,229632.03871061656,445275.99144155625,378583.371085837,342322.67223384685,315314.1962839727,374758.20577928657,1124942.7514154406,1570218.7428569968,1948802.1139428339,2291124.7861766806,2606438.9824606534,0.0,0.0,0.9532971725566067
2019-03-01,PaidSearchNonBranded,4535949.2,1307633.4962345588,1433957.1419366465,3281199.3955640015,2502662.1769004525,5726624.166086447,1500748.6762217423,1369577.8091439442,2619232.360000001,2390301.9700000007,401290.18142412003,175281.6597742964,277918.68562603806,493624.25168557424,419076.723516858,381107.60304358683,348912.4205771782,179207.3375006481,1033697.8643251027,1527322.1160106768,1946398.8395275348,2327506.4425711217,2676418.8631483,0.0,0.0,0.9119055639755872
2019-03-01,PaidSearchBranded,979559.98,1178133.6870773537,1339745.3147172616,3498539.0769681265,2338235.80061406,6105943.5922270985,2525319.7716887947,8693880.488917433,4407399.7,1.5173288819999997E7,341931.6984430309,152337.9469182246,233114.83849392075,414268.2366555126,353517.85300339677,322854.2353268722,295667.41367192415,3858724.411685388,4586108.895540564,5000377.132196077,5353894.985199474,5676749.220526346,5972416.63419827,0.0,0.0,0.8793713806164613
2019-04-01,PaidSearchBranded,1077530.85,1150603.6823407004,1252118.12130694,3592183.3074491387,2162506.2269474063,6203982.38673126,2505514.4335972415,8871418.88098777,4327219.990000002,1.532163639E7,322124.97720485466,154067.28379300496,205923.898217401,399906.6924783204,341647.82433401665,310575.9113342689,286191.85539160157,3766770.5787560786,4448886.737971339,4848793.43044966,5190441.254783677,5501017.166117946,5787209.021509547,0.0,0.0,0.9189258287706271
2019-03-01,Affiliate,405097.52844509156,343367.7437818635,471970.1000750351,1517772.3764386324,823721.7721099069,2648943.548289884,627053.707172703,4095584.677678856,1094386.6800000002,7147957.609999998,106066.97442161535,46207.759134461405,72472.5454450317,129366.67920945652,109935.30759723838,100061.79310202626,91344.31772965693,1584644.948068576,1809392.2270696843,1938758.9062791409,2048694.2138763792,2148756.0069784056,2240100.3247080627,0.0,0.0,0.727520119870462
2019-04-01,Affiliate,321518.16955020826,346674.8068974481,461085.24156950746,1566643.3180218483,796330.3853528235,2705715.916874986,660195.1107271686,4357887.744075296,1140208.75,7526414.0200000005,102421.97382630341,48865.82722136857,66289.99398926625,128915.10685997087,109844.29305671691,99353.40075375338,91379.27341774278,1706210.3900411476,1923788.1850780859,2052703.2919380567,2162547.5849947734,2261900.9857485266,2353280.2591662696,0.0,0.0,0.7518670641406503
2019-03-01,TV_Radio,4773041.265443803,380332.1654063802,349004.53386944975,711590.778990501,609111.9608374141,1241927.8623665702,null,null,null,null,101636.42123297714,46835.12720356262,68998.60782045753,122529.44544909311,104828.4842117262,95912.64714420018,88075.02230388613,-3712445.9525838518,-3494975.7963268547,-3372446.350877762,-3267617.8666660357,-3171705.2195218354,-3083630.197217949,60.0,60.0,1.0897628211003385
2019-04-01,TV_Radio,4568243.753378187,445386.2934379365,393577.47944199387,746816.302588138,679739.1840245856,1289810.3439689714,null,null,null,null,115654.94203058662,56901.206310682945,74769.5689843089,146316.46877414995,124879.45217646423,113536.67691308646,104840.5009330

In [28]:
channel_dna.dal.snowflake.table_from_df(
  df = df
  , database = 'sandbox'
#   , schema = 'cbpm'
  , schema = 'public'
  , name = 'rswaddle_US_mmm_overview_output_test'
  , mode = 'overwrite'
)

In [29]:
query = """
select *
from rswaddle_US_mmm_pdt_channel_output_test
"""
df_pdt_channel_output = channel_dna.dal.databricks.execute_reader(query=query)
display(df_pdt_channel_output)

month_start_date,product_category,channel,paid_avg_fo_agp,baseline_avg_m1_m3_rpt_agp,baseline_avg_m4_m6_rpt_agp,baseline_avg_m7_m12_rpt_agp,baseline_avg_m13_m24_rpt_agp,baseline_avg_m25_m36_rpt_agp,baseline_avg_m37_m48_rpt_agp,baseline_avg_m49_m60_rpt_agp
2019-03-01,Packaging & Long Tail,PaidSearchNonBranded,153860.27314302305,35766.75550271903,18422.191958549352,29110.097646577407,49036.03242474192,43955.05224912923,42091.063920153334,39905.6215946815
2019-03-01,Packaging & Long Tail,TV_Radio,28546.07453013204,6635.8940318047,3417.914539696081,5400.867943515032,9097.775583418226,8155.08884683394,7809.258512106922,7403.788027557395
2019-03-01,Packaging & Long Tail,Affiliate,28335.73569948338,6586.998125312028,3392.7299859747686,5361.07220044263,9030.739554499429,8094.998908721142,7751.716789446043,7349.233972692755
2019-03-01,Packaging & Long Tail,PaidSearchBranded,122999.56843213314,28592.796575647797,14727.139203562727,23271.30567497396,39200.57271873395,35138.7161000885,33648.599416014724,31901.504747747244
2019-03-01,Packaging & Long Tail,Display,31788.217492898206,7389.570938034585,3806.10688328836,6014.27578624358,10131.06263146647,9081.309504164132,8696.201216005627,8244.679100899892
2019-04-01,Packaging & Long Tail,Display,31656.650969878905,6976.505891689031,4060.176251505703,5913.617337763328,10215.072026400714,9213.245969101326,8880.361876162286,8542.691335611606
2019-04-01,Packaging & Long Tail,TV_Radio,31462.568492934945,6933.733914793514,4035.283881669391,5877.361781175743,10152.444855796623,9156.760853222248,8825.917625861639,8490.31729592684
2019-04-01,Packaging & Long Tail,PaidSearchNonBranded,143649.07212852844,31657.442191023245,18423.949891668282,26834.349732735805,46353.15402487639,41807.14618272752,40296.610809843674,38764.3558697555
2019-04-01,Packaging & Long Tail,PaidSearchBranded,121144.3591154406,26697.844188190706,15537.570615173381,22630.35919746999,39091.25937338891,35257.44967026643,33983.561597355765,32691.356642823484
2019-04-01,Packaging & Long Tail,Affiliate,29660.325261506605,6536.554802755415,3804.1341882086163,5540.693924857586,9570.891095240879,8632.24200240962,8320.35018288809,8003.9737578047325


In [30]:
channel_dna.dal.snowflake.table_from_df(
  df = df_pdt_channel_output
  , database = 'sandbox'
#   , schema = 'cbpm'
  , schema = 'public'
  , name = 'rswaddle_US_mmm_pdt_channel_output_test'
  , mode = 'overwrite'
)

In [31]:
query = """
select *
from rswaddle_US_mmm_avg_ltv_agp_test
"""
df_avg_ltv_agp = channel_dna.dal.databricks.execute_reader(query=query)
display(df_avg_ltv_agp)

fo_month_start_date,fo_pi_mktg_category,count_shopper_key,avg_fo_agp,avg_m1_m3_rpt_agp,avg_m4_m6_rpt_agp,avg_m7_m12_rpt_agp,avg_m13_m24_rpt_agp,avg_m25_m36_rpt_agp,avg_m37_m48_rpt_agp,avg_m49_m60_rpt_agp
2017-09-01,Other,9279,5.335765758654993,51.51171864480032,11.467736554444235,19.112005206795235,31.273571244965407,23.13740570331678,19.243003137089623,18.026324217214217
2018-07-01,Marketing Materials,14579,49.12853758054159,26.245441513272773,12.703595857189384,23.094961995653893,32.74300909733514,28.874777505617402,26.349536649805213,21.299488665993817
2020-04-01,Business Cards,64976,26.884539698401063,8.901169275432059,5.04909890161556,9.251160733285838,15.210227897496315,12.566176742064924,11.33359619769777,10.300449338253078
2017-06-01,Business Cards,163897,20.17856788920201,7.40856550666168,4.272888591531198,7.346417868098778,11.601072266733814,8.888629167134875,8.922125761274629,8.276284862883188
2018-08-01,PPAG,8264,24.08743479359582,11.652809049177245,3.929279742933001,6.817649497065535,8.379186263053484,7.530602818040762,6.945824116964535,5.706693177495591
2018-06-01,PPAG,7571,25.883215145588693,10.932631930857209,3.4204153082936055,6.364056365727401,9.008393185116208,7.379817456507659,6.9387249135834,5.5902667379507776
2017-05-01,Business Cards,191747,18.540164663972117,6.845289263101624,4.12780792458474,7.092215197784255,11.490333452691488,9.175259110976102,8.576713734841565,7.937851138304252
2017-08-01,Marketing Materials,15316,52.23237839050256,26.137216828831225,13.763385808720026,26.083404079485597,41.471064764213224,30.11446294047732,27.227390959264312,25.186721233697273
2018-12-01,Packaging & Long Tail,39821,19.898330300757937,4.885123397428892,3.0068414905957535,5.742188967434036,8.6319248200215,7.963020533904516,7.5998496416037655,7.168507653306145
2019-02-01,Business Cards,161708,21.031776748747262,8.033665022747467,4.180689964658804,7.443234046998852,12.083173801469533,10.285927463160483,9.33749411664609,8.53372978309435


In [32]:
channel_dna.dal.snowflake.table_from_df(
  df = df_pdt_channel_output
  , database = 'sandbox'
#   , schema = 'cbpm'
  , schema = 'public'
  , name = 'rswaddle_US_mmm_avg_ltv_agp_test'
  , mode = 'overwrite'
)

In [33]:
query = """
select *
from rswaddle_US_mmm_channel_output_test
"""
df_channel_output = channel_dna.dal.databricks.execute_reader(query=query)
display(df_channel_output)

month_start_date,channel,new_agp_contribution,rpt_agp_contribution,new_bookings_contribution,rpt_bookings_contribution,spend_usd,new_contribution_pct,rpt_contribution_pct
2019-03-01,Display,381321.27304773754,1181783.792166657,665513.8423560119,2062548.113491768,978361.813146,0.04316384000369216,0.0358232677801989
2019-03-01,PaidSearchBranded,1339745.3147172616,3498539.0769681265,2338235.80061406,6105943.5922270985,979559.98,0.15165309805024316,0.10605078782130138
2019-03-01,Momentum,4074203.441457197,1.8614920160443246E7,7110641.284691247,3.2488318687575717E7,0.0,0.46118136573915924,0.5642718016905597
2019-03-01,Email,206169.52774546368,1832288.9951948826,359824.33786071726,3197864.309411671,0.0,0.02333745620357219,0.05554195256412976
2019-03-01,Affiliate,471970.1000750351,1517772.3764386324,823721.7721099069,2648943.548289884,405097.52844509156,0.05342487641284839,0.046008048706495046
2019-03-01,TV_Radio,349004.53386944975,711590.778990501,609111.9608374141,1241927.8623665702,4773041.265443803,0.03950573158455334,0.021570364388702155
2019-03-01,Discount,577904.6030994456,2351184.8896846483,1008607.5446301932,4103484.910550836,0.0,0.06541618207190576,0.07127118044960691
2019-03-01,PaidSearchNonBranded,1433957.1419366465,3281199.3955640015,2502662.1769004525,5726624.166086447,4535949.2,0.16231744993402575,0.0994625965990062
2019-04-01,PaidSearchNonBranded,1233650.1497948235,2998087.2959844633,2130610.591292558,5177931.966778874,3856979.24,0.14040081936792315,0.08688372731184629
2019-04-01,Discount,643629.2957480366,2435713.095608658,1111598.2879060248,4206667.603223571,0.0,0.0732509784133314,0.07058628102397066


In [34]:
channel_dna.dal.snowflake.table_from_df(
  df = df_channel_output
  , database = 'sandbox'
#   , schema = 'cbpm'
  , schema = 'public'
  , name = 'rswaddle_US_mmm_channel_output_test'
  , mode = 'overwrite'
)

In [35]:
query = """
select *
from rswaddle_us_elasticity_output
"""
df_elasticity_output = channel_dna.dal.databricks.execute_reader(query=query)
display(df_elasticity_output)

locale,month_start_date,metric,psnb,psb,display,tv_radio,affiliate
US,2019-03-01,relative_elasticity,0.27896843491098694,0.30327199926913617,0.23606729474851312,0.015312039847706524,0.1663802312236573
US,2019-03-01,spend_shock_agp_roi,0.017886340017492897,0.28408044212099504,0.10330221589848865,-0.6333844549085328,0.18857069344741784
US,2019-04-01,spend_shock_agp_roi,0.03354587704425782,0.422990361150712,0.2650801418046578,-0.3874676777633066,0.26126343452916045
US,2019-04-01,relative_elasticity,0.24412643476720522,0.319984756477317,0.23590022930770813,0.025785780575777652,0.17420279887199197
US,2019-03-01,total_bookings,7.108001531264931E7,7.727246402922334E7,6.014898040666879E7,3901445.0763583905,4.23930018708863E7
US,2019-04-01,total_bookings,5.998178705415639E7,7.862015247099677E7,5.796061100001129E7,6335558.05210682,4.280157204663985E7
US,2019-03-01,total_spend,4.001125444614872E7,3.447993758668559E7,3.1236885147610616E7,6097454.424129594,2.043633879580184E7
US,2019-04-01,total_spend,3.36029631778067E7,3.1990413789638013E7,2.6527898711130746E7,5988855.001257318,1.964906319403201E7
US,2019-03-01,auc_total,0.9737792245739277,1.05861429224964,0.824026658431163,0.05344886526069657,0.5807739954418151
US,2019-04-01,auc_total,0.8022032580424259,1.0514748819184363,0.7751718190783181,0.08473247564845328,0.5724331039712305


In [36]:
channel_dna.dal.snowflake.table_from_df(
  df = df_elasticity_output
  , database = 'sandbox'
#   , schema = 'cbpm'
  , schema = 'public'
  , name = 'rswaddle_US_elasticity_output_test'
  , mode = 'overwrite'
)

In [37]:
query = """
select distinct
affiliate_spend
, display_spend
, figure
, month_start_date
, period
, psb_spend
, psnb_total_spend_usd as psnb_spend
, response
, tv_spend
from rswaddle_us_channel_interactions
where
response in ('display_clicks','psb_clicks','psnb_total_clicks')
"""
df_ci_output = channel_dna.dal.databricks.execute_reader(query=query)
display(df_ci_output)

affiliate_spend,display_spend,figure,month_start_date,period,psb_spend,psnb_spend,response,tv_spend
648.7096189042172,566.5677375027861,value,2019-04-01,6,1179.2556228803612,2060.1295197497398,psnb_total_clicks,1050.4401363541795
-264.6457444707186,59.264161986486236,value,2019-03-01,18,-334.0133609447669,37.48962358220238,display_clicks,-96.16547785144304
-269.79939960194076,227.38002958354573,value,2019-04-01,14,-233.5048486601809,237.5173968150804,display_clicks,84.27529882161582
424.622367886925,443.1767854531799,value,2019-03-01,8,728.8366945218997,328.7317881428688,psb_clicks,695.8726104560006
1111.2429964244814,838.3797836866385,value,2019-03-01,3,2008.6944395838536,1628.5377869268846,psb_clicks,1132.6854388157599
-231.6975213343461,453.1278094717338,value,2019-04-01,11,-170.79729128601326,386.56978285808947,display_clicks,192.04506322196727
721.1368259236053,593.1918687074968,value,2019-03-01,5,1282.651211391717,883.7388217957572,psb_clicks,1002.3451143790636
332.5349390422702,332.6068010911308,value,2019-04-01,13,617.3098406372593,1265.5131548002466,psnb_total_clicks,675.3864358487864
1700.4529463819422,1106.8276834739574,value,2019-04-01,1,3248.520338827241,2584.653202043419,psb_clicks,1193.4724612030886
123.84166083106703,271.60443792728125,value,2019-03-01,17,150.20762004497948,-134.95302420608903,psb_clicks,116.77061510579487


In [38]:
channel_dna.dal.snowflake.table_from_df(
  df = df_ci_output
  , database = 'sandbox'
#   , schema = 'cbpm'
  , schema = 'public'
  , name = 'rswaddle_US_channel_interactions_test'
  , mode = 'overwrite'
)